<a href="https://colab.research.google.com/github/mike1336git/colab_notebook/blob/main/with_js/js100_electronsTDKS3D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### simulator( html + css + js ) + control( python )

In [1]:
#@title js100_electronsTDKS3D / def exec_html_js() ... exec me first
#
#  Copyright(C) 2023-2024 Mitsuru Ikeuchi
#  home page: https://mike1336.web.fc2.com/index.html
#  Released under the MIT license ( https://opensource.org/licenses/MIT )
#
#  ver 0.0.0  2023.12.06 created,  last updated on 2024.03.20
#

# def exec_html_js()

import IPython
from IPython.display import display, HTML
from google.colab.output import eval_js

def exec_html_js():
  htm = HTML('''


<!DOCTYPE html>
<html>
<head>
<meta charset="utf-8">
<title>js100_electronsTDKS3D</title>
<script type="text/javascript">

// %%%%%%%%%%%%%%%%%%%%  javaScript  %%%%%%%%%%%%%%%%%%%%

'use strict';

/* --------------------
//
//  js100_electronsTDKS3D
//    Copyright(C) 2018-2023 Mitsuru Ikeuchi
//    Released under the MIT license ( https://opensource.org/licenses/MIT )
//
//    ver 0.0.0  2018.03.31 created, last updated on 2018.12.06
//    ver 0.0.1  2019.01.26 v1, last updated on 2021.09.22
//    ver 0.0.2  2021.11.06 v2, last updated on 2021.11.06
//    ver 0.0.3  2023.05.11 v3, last updated on 2023.09.09
//
//
// -------------------- TDKS3D: time dependent Kohn-Sham 3D : LDA + spin
//
//  time dependent Kohn-Sham 3D : LDA + spin
//
//   - solve time dependent Kohn-Sham equation : approximation
//       N. Watanabe and M. Tsukada: Phys. Rev., E65, 036705 (2002)
//   - Vxc: LDA
//       J. P. Perdew and A. Zunger; Phys. Rev., B23, 5048 (1981)
//
//  many electron system --> one electron approximation
//    one electron ,  others -> consider as effective potential Veff
//    Kohn-Sham equation {(-d^2/dr^2)+Veff(r)} |i> = e_i |i>,  Veff(r)=Vext+VH+Vx+Vc
//      Veff: one electron feels effective potential
//      Vext: external Potential
//      Vh  : Hartree potential (electron charge -> electro-static potential)
//      Vx,Vc :electron exchange and correlation potential (LDA:Perdew-Zunger)
//    successive approximation
//      solve Kohn-Sham -> electron state,occupation -> electron charge density -> Veff
//
//   quantum electron dynamics
//   - real-space : sttate psi(i) = ph(i)*si (si = spinUp or spinDown)
//   - time dependent Kohn-Sham equation :
//      N. Watanabe and M. Tsukada: Physical Review E, Vol 65, No 3, 036705 (2002)
//      (arXiv physics/0112015)
//      H ph(ri,t) = i d ph(ri,t)/dt, H = -D2/2 + Veff
//      ph(ri,t+dt) = exp(-idt*H) ph(ri,t)
//      approximation :
//      ph(ri,t+dt) = exp(idt/2*D/2)exp(-idt*Veff)exp(idt/2*D/2)ph(ri,t)
//      Veff: one electron feels effective potential
//        Veff = Vext + Vh + Vx + Vc
//          Vx + Vc : LDA( J. P. Perdew and A. Zunger; Phys. Rev., B23, 5048 (1981) )
//          Vext: external Potential
//          Vh  : Hartree potential (electron charge -> electro-static potential)
//          Vx,Vc :electron exchange and correlation potential (LDA:Perdew-Zunger)
//  loss process
//      ph(i) steepest descent method: |ph(next)> = |ph(i)> - dump{H-Ei}|ph(i)>
//      spin : as another degree of freedom of state : psi(i) = ph(i)*si
//      <psi(i)|psi(j)> = 0 (si,sj parallel), or = 0 (<ph(i)|ph(j)> = 0)
//      {psi(0),..,psi(N)} Gram-Schmidt orthogonallization
//
//  procedure : evolve time dependent Kohn-Sham equation
//      ( state orthogonallity conserved in this process )
//
//   (1) set initial orbit
//      set external potential Vext
//      set initial orbit ph(i)
//
//   (2) evolve time dt/2
//      t <-- t + dt/2
//
//   (3) evolve kinetic part dt/2 - exp(idt/2*D/2)*
//      krStep(ph(i),0.5*dt);
//
//   (4) set electron density
//      rho <-- sum(|ph(i)|^2,i)
//
//   (5) set effective potential
//      Veff = Vext + Vh + Vx + Vc
//      Vh <-- rho (Poisson eq. ,SOR iteration)
//      Vx,Vc <-- rho (LDA:Perdew-Zunger)
//
//   (6) evolve potential part dt - exp(-idt*Veff)*
//      phaseStep(ph(i), Veff, dt);
//
//   (7) evolve time dt/2
//      t <-- t + dt/2
//
//   (8) evolve kinetic part dt/2 - exp(idt/2*D/2)*
//      krStep(ph(i,t+dt/2),0.5*dt);
//
//   goto (2)
//
// --------------------
*/

const electronsTDKS3D = (function(){ // ====================  electronsTDKS3D Module  ====================

	// (au): atomic unit hBar=1,e=1,me=1,a0=1
	const g_auLength = 5.29177211e-11;					// (m) 1(au) = g_auLength (m)
	const g_auTime = 2.418884326e-17;					// (s) 1(au) = g_auTime (s)
	const g_auEnergy = 4.35974465e-18;					// (J) 1(au) = g_auEnergy (J)
	const g_au2eV = 27.211386;							// (eV) 1(au) = 27.211386 (eV)
	const g_nxMax = 64;									// maximum number of g_NNx
	const g_nyMax = 64;									// maximum number of g_NNy
	const g_nzMax = 64;									// maximum number of g_NNz
	const g_nMax = 64;									// max( g_NNx,g_NNy,g_NNz)
	const g_nOrbitMax = 6;								// maximum number of orbit
	const g_nStateMax = g_nOrbitMax*2;					// maximum number of state |orbit>,spin

	let g_iterCount = 0;								// iteration count
	let g_sysTime = 0.0;								// (au) system time
	let g_dx = 1.0/2.0;									// (au) x-division
	let g_dy = g_dx;									// (au) y-division
	let g_dz = g_dy;									// (au) z-division
	let g_timeStep = 0.5*( g_dx*g_dx );					// (au) time step dt
	let g_NNx = 32;										// number of space x-division
	let g_NNy = g_NNx;									// number of space y-division
	let g_NNz = g_NNx;									// number of space z-division
	let g_numberOfOrbit = 6;							// number of orbit
	let g_numberOfState = g_numberOfOrbit*2;			// number of state |orbit>,spin
	let g_numberOfElectron = 6.0;						// number of electron
	let g_dampingFactor = 0.1*g_dx*g_dx;				// damping factor of steepest descent method : damp = a*dx*dx
	//let g_mixing = 0.5;								// charge mixing in setRho()
	let g_broadening = 0.05;							// (au) level broadening in setOccupation()

	const g_psis = dimInt2( g_nStateMax, 2 );			// state |orbit>,S: g_psis[orbit][spin]
	const g_occ = dim1( g_nStateMax );					// occupation: g_occ[state] 0.0 ... 1.0
	const g_energy = dim1( g_nStateMax );				// (au) state energy: g_energy[state]
	const g_psi = dim5( g_nOrbitMax, g_nxMax, g_nyMax, g_nzMax, 2 );
					// orbit g_psi[orbit][ix][iy][iz][0/1]  0:real part of the orbit, 1:imaginary part of the orbit
	const g_wrk = dim4( g_nxMax, g_nyMax, g_nzMax, 2 );	// work orbit g_wrk[ix][iy][iz][0/1]
	const g_vv = dim3( g_nxMax, g_nyMax, g_nzMax );		// (au) effective potential g_vv[ix][iy][iz]
	const g_vvext = dim3( g_nxMax, g_nyMax, g_nzMax );	// (au) externalpotential g_vvext[ix][iy][iz]
	const g_vvh = dim3( g_nxMax, g_nyMax, g_nzMax );	// (au) Hartree potential g_vvh[ix][iy][iz]
	const g_vvx = dim3( g_nxMax, g_nyMax, g_nzMax );	// (au) exchange potential g_vvx[ix][iy][iz]
	const g_vvc = dim3( g_nxMax, g_nyMax, g_nzMax );	// (au) correlation potential g_vvc[ix][iy][iz]
	const g_rho = dim3( g_nxMax, g_nyMax, g_nzMax );	// (au) charge density g_rho[ix][iy][iz]

	const g_bRe = dim1( g_nMax );						// work b vector in kxStep(),kyStep(),kzStep()
	const g_bIm = dim1( g_nMax );						// work b vector in kxStep(),kyStep(),kzStep()
	const g_uRe = dim1( g_nMax );						// work u vector in kxStep(),kyStep(),kzStep()
	const g_uIm = dim1( g_nMax );						// work u vector in kxStep(),kyStep(),kzStep()

	// QMD
	const g_nucNNMax = 8;								// nuc array max
	let g_nucMax = 2;									// maximum number of nucleus
	let g_jelliumRadius = 2.0;							// nuc potential jellium radius
	const g_nucMass = dim1( g_nucNNMax );				// mass of nucleus
	const g_nucCharge = dim1( g_nucNNMax );				// charge of nucleus
	const g_nucxx = dim1( g_nucNNMax );					// (au) nuclear x-position
	const g_nucyy = dim1( g_nucNNMax );					// (au) nuclear y-position
	const g_nuczz = dim1( g_nucNNMax );					// (au) nuclear z-position
	const g_nucvx = dim1( g_nucNNMax );					// (m/s) nuclear x-velocity
	const g_nucvy = dim1( g_nucNNMax );					// (m/s) nuclear y-velocity
	const g_nucvz = dim1( g_nucNNMax );					// (m/s) nuclear z-velocity
	const g_nucfx = dim1( g_nucNNMax );					// (N) nuclear x-force
	const g_nucfy = dim1( g_nucNNMax );					// (N) nuclear y-force
	const g_nucfz = dim1( g_nucNNMax );					// (N) nuclear z-force

	function dim1( n ) {
		return new Float64Array( n );
	}

	function dimInt2( ni, nj ) {
		let a = [];
		for (let i=0; i<ni; i++) {
			a[i] = new Int32Array( nj );
		}
		return a;
	}

	function dim3( ni, nj, nk ) {
		let a = [];
		for (let i=0; i<ni; i++) {
			a[i] = [];
			for (let j=0; j<nj; j++) {
				a[i][j] = new Float64Array( nk );
			}
		}
		return a;
	}

	function dim4( ni, nj, nk, nl ) {
		let a = [];
		for (let i=0; i<ni; i++) {
			a[i] = [];
			for (let j=0; j<nj; j++) {
				a[i][j] = [];
				for (let k=0; k<nk; k++) {
					a[i][j][k] = new Float64Array( nl );
				}
			}
		}
		return a;
	}

	function dim5( ni, nj, nk, nl, nm ) {
		let a = [];
		for (let i=0; i<ni; i++) {
			a[i] = [];
			for (let j=0; j<nj; j++) {
				a[i][j] = [];
				for (let k=0; k<nk; k++) {
					a[i][j][k] = [];
					for (let l=0; l<nl; l++) {
						a[i][j][k][l] = new Float64Array( nm );
					}
				}
			}
		}
		return a;
	}


	// --------------------  set initial condition  --------------------

	function setInitialCondition( theme ) {
		let vIndex = 0;

		g_iterCount = 0;
		g_sysTime = 0.0;
		setInitialState(theme);
		setExternalPotential(vIndex);
	}

	function setInitialState(phIndex) {
		const xmax=g_NNx*g_dx, ymax=g_NNy*g_dy, zmax=g_NNz*g_dz;

		if (phIndex==0) { // 2-electron up,up
			g_numberOfOrbit = 2;
			g_numberOfState = g_numberOfOrbit*2;
			g_numberOfElectron =  2.0;
			//setGaussianWaveAndSpin(iOrbit, xPos, yPos,   zPos,      width, kx, ky,  kz,  spin)
			setGaussianWaveAndSpin(0, xmax*0.25, ymax*0.5 , zmax*0.5 , 1.0, 0.0, 2.0, 0.0,  1); // 1:up
			setGaussianWaveAndSpin(1, xmax*0.5 , ymax*0.5 , zmax*0.5 , 1.0, 0.0, 0.0, 0.0,  1); // 1:up
		} else if (phIndex==1) { // 2-electron up,down
			g_numberOfOrbit = 2;
			g_numberOfState = g_numberOfOrbit*2;
			g_numberOfElectron =  2.0;
			setGaussianWaveAndSpin(0, xmax*0.25, ymax*0.5 , zmax*0.5 , 1.2, 0.0, 2.0, 0.0,  1); // 1:up
			setGaussianWaveAndSpin(1, xmax*0.75, ymax*0.5 , zmax*0.5 , 1.2, 0.0,-2.0, 0.0, -1); //-1:down
		} else if (phIndex==2) { // 3-electron up,up,down
			g_numberOfOrbit = 3;
			g_numberOfState = g_numberOfOrbit*2;
			g_numberOfElectron =  3.0;
			setGaussianWaveAndSpin(0, xmax*0.25, ymax*0.5 , zmax*0.5 , 1.2, 0.0, 2.0, 0.0,  1); // 1:up
			setGaussianWaveAndSpin(1, xmax*0.75, ymax*0.5 , zmax*0.5 , 1.2, 0.0,-2.0, 0.0,  1); // 1:up
			setGaussianWaveAndSpin(2, xmax*0.5 , ymax*0.5 , zmax*0.5 , 1.2, 0.0, 0.0, 0.0, -1); //-1:down
		} else if (phIndex==3) { // 4-electron up,up,up,up
			g_numberOfOrbit = 4;
			g_numberOfState = g_numberOfOrbit*2;
			g_numberOfElectron =  4.0;
			setGaussianWaveAndSpin(0, xmax*0.25, ymax*0.5 , zmax*0.5 , 1.2, 0.0, 2.0, 0.0,  1); // 1:up
			setGaussianWaveAndSpin(1, xmax*0.75, ymax*0.5 , zmax*0.5 , 1.2, 0.0,-2.0, 0.0,  1); // 1:up
			setGaussianWaveAndSpin(2, xmax*0.5 , ymax*0.5 , zmax*0.5 , 1.2, 0.0, 0.0, 0.0,  1); // 1:up
			setGaussianWaveAndSpin(3, xmax*0.5 , ymax*0.75, zmax*0.5 , 1.2, 1.0, 0.0, 0.0,  1); // 1:up
		} else if (phIndex==4) { // 1-electron up
			g_numberOfOrbit = 1;
			g_numberOfState = g_numberOfOrbit*2;
			g_numberOfElectron =  1.0;
			setGaussianWaveAndSpin(0, xmax*0.5 , ymax*0.5 , zmax*0.5 , 1.0, 0.0, 0.0, 0.0,  1); // 1:up
		}
	}

	function setExternalPotential(vIndex) {
		if (vIndex==0) {
			const xPos = 0.5*g_NNx*g_dx, yPos = 0.5*g_NNy*g_dy, zPos = 0.5*g_NNz*g_dz;
			const vvextAtXMax = 8.0; // (au)
			setHarmonicPotential(xPos, yPos, zPos, vvextAtXMax)
		}
	}

	// electron states

	function setGaussianWaveAndSpin(iOrbit, xPos, yPos, zPos, width, kx, ky, kz, spin) {
		setGaussianWave(g_psi[iOrbit], xPos, yPos, zPos, width, kx, ky, kz);
		setElectronStates(iOrbit, spin);
	}

	function setHOrbitalWaveAndSpin(iOrbit, xPos, yPos, zPos, kx, ky, kz, state, spin) {
		setHOrbitalWave(g_psi[iOrbit], xPos, yPos, zPos, kx, ky, kz, state);
		setElectronStates(iOrbit, spin);
	}

	function setElectronStates(iOrbit, spin) {
		// set electron state orbit -> g_psis[state][0], spin -> g_psis[state][1], and g_occ[]
		const iState = iOrbit*2;
		g_psis[iState][0] = iOrbit; g_psis[iState][1] = 1;
		g_psis[iState+1][0] = iOrbit; g_psis[iState+1][1] = -1;
		if (spin==1) { // 1-up spin
			g_occ[iState] = 1.0;
			g_occ[iState+1] = 0.0;
		} else if (spin==-1) { // -1-down spin
			g_occ[iState] = 0.0;
			g_occ[iState+1] = 1.0;
		} else if (spin==2) { // 2-up and down spin
			g_occ[iState] = 1.0;
			g_occ[iState+1] = 1.0;
		}
	}

	// --- wave library

	function setGaussianWave(ph, xPos, yPos, zPos, waveWidth, kx, ky, kz) { // ph[][][][]
		const nnx=g_NNx, nny=g_NNy, nnz=g_NNz;

		for (let i=1; i<nnx-1; i++) {
			for (let j=1; j<nny-1; j++) {
				for (let k=1; k<nnz-1; k++) {
					const x = i*g_dx, y = j*g_dy, z = k*g_dz;
					const r2 = (x-xPos)*(x-xPos)+(y-yPos)*(y-yPos)+(z-zPos)*(z-zPos);
					const phAb = Math.exp(-r2/(4.0*waveWidth*waveWidth));
					const phPh = kx*x+ky*y+kz*z;
					ph[i][j][k][0] = phAb*Math.cos(phPh);
					ph[i][j][k][1] = phAb*Math.sin(phPh);
				}
			}
		}
		setWaveBoundary(ph);
		normalize(ph);
	}

	function setHOrbitalWave(ph, xPos, yPos, zPos, kx, ky, kz, state) { // ph[][][][]
		const nnx=g_NNx, nny=g_NNy, nnz=g_NNz;

		for (let i=1; i<nnx-1; i++) {
			for (let j=1; j<nny-1; j++) {
				for (let k=1; k<nnz-1; k++) {
					const x = i*g_dx, y = j*g_dy, z = k*g_dz;
					const r = Math.sqrt((x-xPos)*(x-xPos)+(y-yPos)*(y-yPos)+(z-zPos)*(z-zPos));

					let phAb = 0.0;
					if (state==0) { // 1s
						phAb = Math.exp( -r );
					} else if (state==1) { // 2s
						phAb = (1.0-0.5*r)*Math.exp( -r/2.0 );
					} else if (state==2) { // 2px
						phAb = (x-xPos)*Math.exp( -r/2.0 );
					} else if (state==3) { // 2py
						phAb = (y-yPos)*Math.exp( -r/2.0 );
					} else if (state==4) { // 2pz
						phAb = (z-zPos)*Math.exp( -r/2.0 );
					} else if (state==5) { // 3s
						phAb = (1.0-(2.0/3.0)*r+(2.0/27.0)*r*r)*Math.exp( -r/3.0 );
					} else if (state==6) { // 3px
						phAb = (x-xPos)*(1.0-(1.0/6.0)*r)*Math.exp( -r/3.0 );
					} else if (state==7) { // 3py
						phAb = (y-yPos)*(1.0-(1.0/6.0)*r)*Math.exp( -r/3.0 );
					} else if (state==8) { // 3pz
						phAb = (z-zPos)*(1.0-(1.0/6.0)*r)*Math.exp( -r/3.0 );
					} else if (state==9) { // 3dex
						phAb = (y-yPos)*(z-zPos)*Math.exp( -r/3.0 );
					} else if (state==10) { // 3dey
						phAb = (z-zPos)*(x-xPos)*Math.exp( -r/3.0 );
					} else if (state==11) { // 3dez
						phAb = (x-xPos)*(y-yPos)*Math.exp( -r/3.0 );
					} else if (state==12) { // 3drz
						phAb = (2.0*(z-zPos)*(z-zPos)-(x-xPos)*(x-xPos)-(y-yPos)*(y-yPos))*Math.exp( -r/3.0 );
					} else if (state==13) { // 3drrz
						phAb = ((x-xPos)*(x-xPos)-(y-yPos)*(y-yPos))*Math.exp( -r/3.0 );
					}
					const phPh = kx*x+ky*y+kz*z;
					ph[i][j][k][0] = phAb*Math.cos(phPh);
					ph[i][j][k][1] = phAb*Math.sin(phPh);
				}
			}
		}
		setWaveBoundary(ph);
		normalize(ph);
	}

	function setWaveBoundary(ph) { // ph[][][][]
		const nnx=g_NNx, nny=g_NNy, nnz=g_NNz;

		for (let i=0; i<nnx; i++) {
			for (let j=0; j<nny; j++) {
				ph[i][j][0][0] = 0.0;
				ph[i][j][0][1] = 0.0;
				ph[i][j][nnz-1][0] = 0.0;
				ph[i][j][nnz-1][1] = 0.0;
			}
		}
		for (let i=0; i<nnx; i++) {
			for (let k=0; k<nnz; k++) {
				ph[i][0][k][0] = 0.0;
				ph[i][0][k][1] = 0.0;
				ph[i][nny-1][k][0] = 0.0;
				ph[i][nny-1][k][1] = 0.0;
			}
		}
		for (let j=0; j<nny; j++) {
			for (let k=0; k<nnz; k++) {
				ph[0][j][k][0] = 0.0;
				ph[0][j][k][1] = 0.0;
				ph[nnx-1][j][k][0] = 0.0;
				ph[nnx-1][j][k][1] = 0.0;
			}
		}
	}

	// --- potential library

	function setHarmonicPotential(xPos, yPos, zPos, potAtXMax) {
		const nnx=g_NNx,nny=g_NNy,nnz=g_NNz;

		const k0 = potAtXMax/(nnx*g_dx*nnx*g_dx/4.0);
		for (let i=0; i<nnx; i++) {
			for (let j=0; j<nny; j++) {
				for (let k=0; k<nnz; k++) {
					const x = i*g_dx, y = j*g_dy, z = k*g_dz;
					g_vvext[i][j][k] = k0*((x-xPos)*(x-xPos)+(y-yPos)*(y-yPos)+(z-zPos)*(z-zPos));
				}
			}
		}
	}

	function setJelliumPotential(xPos, yPos, zPos, radius, charge) {
		const nnx=g_NNx, nny=g_NNy, nnz=g_NNz;

		for (let i=0; i<nnx; i++) {
			for (let j=0; j<nny; j++) {
				for (let k=0; k<nnz; k++) {
					const x = i*g_dx, y = j*g_dy, z = k*g_dz;
					const r = Math.sqrt((x-xPos)*(x-xPos)+(y-yPos)*(y-yPos)+(z-zPos)*(z-zPos));
					if (r>=radius) {
						g_vvext[i][j][k] = -charge/r;
					} else {
						g_vvext[i][j][k] = -(charge/radius)*(1.5-0.5*r*r/(radius*radius));
					}
				}
			}
		}
	}


	// --------------------  time evolution  --------------------

	function timeEvolution( nTimes, lossSW, qmdSW ) {
		// lossSW 0-lossOFF 1-lossON 2-loss+trans 3-loss+trans,t-stop
		// qmdSW  0-OFF 1-qmd ON (move nuclear)
		const nst=g_numberOfState;

		g_iterCount += 1;
		if (lossSW<3) {
			for (let i=0; i<nTimes; i++) {
				evolveTimeStep(qmdSW,g_timeStep);
			}
		}

		for (let iState=0; iState<nst; iState++) {
			const iOrbit = g_psis[iState][0];
			g_energy[iState] = kineticEnergy(g_psi[iOrbit]) + potentialEnergy(g_psi[iOrbit],g_vv);
		}
		sortState(nst);

		if (lossSW==1 || lossSW==2 || lossSW==3) { // SD loss process
			setLoss(g_dampingFactor);
			GramSchmidt(nst);
			sortState(nst);
			if (lossSW==2 || lossSW==3) { // transition
				setOcc(nst,g_numberOfElectron);
			}
		}
	}

	// --- evolve timeStep

	function evolveTimeStep(qmdSW,dt) {
		const norb=g_numberOfOrbit;

		// (2) evolve time dt/2
		g_sysTime += 0.5*dt;

		// (3) evolve kinetic part dt/2 : exp(idt/2*D/2)*
		for (let iOrbit=0; iOrbit<norb; iOrbit++) {
			kxStep(g_psi[iOrbit],0.5*dt);
			kyStep(g_psi[iOrbit],0.5*dt);
			kzStep(g_psi[iOrbit],0.5*dt);
		}

		// (4) set electron density
		setElectronDensity(g_numberOfState);

		// (5) set effective potential
		setEffectivePotential();

		// (6) evolve potential part dt - exp(-idt*Veff)*
		for (let iOrbit=0; iOrbit<norb; iOrbit++) {
			phaseStep(g_psi[iOrbit],g_vv,dt);
		}

		// (7) evolve time dt/2
		g_sysTime += 0.5*dt;

		// (8) evolve kinetic part dt/2 : exp(idt/2*D/2)*
		for (let iOrbit=0; iOrbit<norb; iOrbit++) {
			kxStep(g_psi[iOrbit],0.5*dt);
			kyStep(g_psi[iOrbit],0.5*dt);
			kzStep(g_psi[iOrbit],0.5*dt);
		}

		if (qmdSW==1) moveNuc(dt);
	}


	// --- (3),(8) kr step

	function kxStep(ph, deltat) { // ph[][][][]
		const nnx=g_NNx, nny=g_NNy, nnz=g_NNz;
		const bRe=g_bRe, bIm=g_bIm, uRe=g_uRe, uIm=g_uIm;

		const a = 4.0*g_dx*g_dx/deltat;
		const bbRe = 2.0;
		const bbIm = a;
		const aaRe = -2.0;
		const aaIm = a;
		const aaAb = aaRe*aaRe+aaIm*aaIm;

		for (let k=1; k<nnz-1; k++) {
			for (let j=1; j<nny-1; j++) {
				for (let i=1; i<nnx-1; i++) {
					bRe[i] = bbRe*ph[i][j][k][0]-bbIm*ph[i][j][k][1] - ph[i+1][j][k][0] - ph[i-1][j][k][0];
					bIm[i] = bbRe*ph[i][j][k][1]+bbIm*ph[i][j][k][0] - ph[i+1][j][k][1] - ph[i-1][j][k][1];
				}

				uRe[1] = aaRe/aaAb;
				uIm[1] = -aaIm/aaAb;
				ph[1][j][k][0] = bRe[1]*uRe[1] - bIm[1]*uIm[1];
				ph[1][j][k][1] = bIm[1]*uRe[1] + bRe[1]*uIm[1];

				for (let i=2; i<nnx-1; i++) {
					const auAb = (aaRe-uRe[i-1])*(aaRe-uRe[i-1])+(aaIm-uIm[i-1])*(aaIm-uIm[i-1]);
					uRe[i] = (aaRe-uRe[i-1])/auAb;
					uIm[i] = -(aaIm-uIm[i-1])/auAb;
					ph[i][j][k][0] = (bRe[i]-ph[i-1][j][k][0])*uRe[i] - (bIm[i]-ph[i-1][j][k][1])*uIm[i];
					ph[i][j][k][1] = (bRe[i]-ph[i-1][j][k][0])*uIm[i] + (bIm[i]-ph[i-1][j][k][1])*uRe[i];
				}

				for (let i=nnx-3; i>=1; i--) {
					ph[i][j][k][0] -= ph[i+1][j][k][0]*uRe[i] - ph[i+1][j][k][1]*uIm[i];
					ph[i][j][k][1] -= ph[i+1][j][k][0]*uIm[i] + ph[i+1][j][k][1]*uRe[i];
				}
			}
		}
	}

	function kyStep(ph, deltat) { // ph[][][][]
		const nnx=g_NNx,nny=g_NNy,nnz=g_NNz;
		const bRe=g_bRe, bIm=g_bIm, uRe=g_uRe, uIm=g_uIm;

		const a = 4.0*g_dx*g_dx/deltat;
		const bbRe = 2.0;
		const bbIm = a;
		const aaRe = -2.0;
		const aaIm = a;
		const aaAb = aaRe*aaRe+aaIm*aaIm;

		for (let i=1; i<nnx-1; i++) {
			for (let k=1; k<nnz-1; k++) {
				for (let j=1; j<nny-1; j++) {
					bRe[j] = bbRe*ph[i][j][k][0]-bbIm*ph[i][j][k][1] - ph[i][j+1][k][0] - ph[i][j-1][k][0];
					bIm[j] = bbRe*ph[i][j][k][1]+bbIm*ph[i][j][k][0] - ph[i][j+1][k][1] - ph[i][j-1][k][1];
				}

				uRe[1] = aaRe/aaAb;
				uIm[1] = -aaIm/aaAb;
				ph[i][1][k][0] = bRe[1]*uRe[1] - bIm[1]*uIm[1];
				ph[i][1][k][1] = bIm[1]*uRe[1] + bRe[1]*uIm[1];

				for (let j=2; j<nny-1; j++) {
					const auAb = (aaRe-uRe[j-1])*(aaRe-uRe[j-1])+(aaIm-uIm[j-1])*(aaIm-uIm[j-1]);
					uRe[j] = (aaRe-uRe[j-1])/auAb;
					uIm[j] = -(aaIm-uIm[j-1])/auAb;
					ph[i][j][k][0] = (bRe[j]-ph[i][j-1][k][0])*uRe[j] - (bIm[j]-ph[i][j-1][k][1])*uIm[j];
					ph[i][j][k][1] = (bRe[j]-ph[i][j-1][k][0])*uIm[j] + (bIm[j]-ph[i][j-1][k][1])*uRe[j];
				}

				for (let j=nny-3; j>=1; j--) {
					ph[i][j][k][0] -= ph[i][j+1][k][0]*uRe[j] - ph[i][j+1][k][1]*uIm[j];
					ph[i][j][k][1] -= ph[i][j+1][k][0]*uIm[j] + ph[i][j+1][k][1]*uRe[j];
				}
			}
		}
	}

	function kzStep(ph, deltat) { // ph[][][][]
		const nnx=g_NNx, nny=g_NNy, nnz=g_NNz;
		const bRe=g_bRe, bIm=g_bIm, uRe=g_uRe, uIm=g_uIm;

		const a = 4.0*g_dx*g_dx/deltat;
		const bbRe = 2.0;
		const bbIm = a;
		const aaRe = -2.0;
		const aaIm = a;
		const aaAb = aaRe*aaRe+aaIm*aaIm;

		for (let i=1; i<nnx-1; i++) {
			for (let j=1; j<nny-1; j++) {
				for (let k=1; k<nnz-1; k++) {
					bRe[k] = bbRe*ph[i][j][k][0]-bbIm*ph[i][j][k][1] - ph[i][j][k+1][0] - ph[i][j][k-1][0];
					bIm[k] = bbRe*ph[i][j][k][1]+bbIm*ph[i][j][k][0] - ph[i][j][k+1][1] - ph[i][j][k-1][1];
				}

				uRe[1] = aaRe/aaAb;
				uIm[1] = -aaIm/aaAb;
				ph[i][j][1][0] = bRe[1]*uRe[1] - bIm[1]*uIm[1];
				ph[i][j][1][1] = bIm[1]*uRe[1] + bRe[1]*uIm[1];

				for (let k=2; k<nnz-1; k++) {
					const auAb = (aaRe-uRe[k-1])*(aaRe-uRe[k-1])+(aaIm-uIm[k-1])*(aaIm-uIm[k-1]);
					uRe[k] = (aaRe-uRe[k-1])/auAb;
					uIm[k] = -(aaIm-uIm[k-1])/auAb;
					ph[i][j][k][0] = (bRe[k]-ph[i][j][k-1][0])*uRe[k] - (bIm[k]-ph[i][j][k-1][1])*uIm[k];
					ph[i][j][k][1] = (bRe[k]-ph[i][j][k-1][0])*uIm[k] + (bIm[k]-ph[i][j][k-1][1])*uRe[k];
				}

				for (let k=nnz-3; k>=1; k--) {
					ph[i][j][k][0] -= ph[i][j][k+1][0]*uRe[k] - ph[i][j][k+1][1]*uIm[k];
					ph[i][j][k][1] -= ph[i][j][k+1][0]*uIm[k] + ph[i][j][k+1][1]*uRe[k];
				}
			}
		}
	}


	// --- (6) phase step - evolve dt potential part: - exp(-idt*Veff)*

	function phaseStep(ph, v, deltat) { // ph[][][][], v[][][]
		const nnx=g_NNx,nny=g_NNy,nnz=g_NNz;

		for (let i=1; i<nnx-1; i++) {
			for (let j=1; j<nny-1; j++) {
				for (let k=1; k<nnz-1; k++) {
					const th = deltat*v[i][j][k];
					const cs = Math.cos(th), sn = Math.sin(th);
					const phr = ph[i][j][k][0];
					const phi = ph[i][j][k][1];
					ph[i][j][k][0] = cs*phr + sn*phi;
					ph[i][j][k][1] = cs*phi - sn*phr;
				}
			}
		}
	}

	// --- (4) set electron density

	function setElectronDensity(istateMax) {
		const nnx=g_NNx, nny=g_NNy, nnz=g_NNz, psi=g_psi, rho=g_rho;

		for (let i=1; i<nnx-1; i++) {
			for (let j=1; j<nny-1; j++) {
				for (let k=1; k<nnz-1; k++) {
					rho[i][j][k] = 0.0;
					for (let iState=0; iState<istateMax; iState++) {
						const iOrbit = g_psis[iState][0];
						rho[i][j][k] += g_occ[iState]*(psi[iOrbit][i][j][k][0]*psi[iOrbit][i][j][k][0]
										+psi[iOrbit][i][j][k][1]*psi[iOrbit][i][j][k][1]);
					}
				}
			}
		}
	}

	// --- (5) set effective potential

	function setEffectivePotential() {
		const nnx=g_NNx, nny=g_NNy, nnz=g_NNz;

		poisson(20); // set VH
		setVxc();

		for (let i=0; i<nnx; i++) {
			for (let j=0; j<nny; j++) {
				for (let k=0; k<nnz; k++) {
					g_vv[i][j][k] = g_vvext[i][j][k] + g_vvh[i][j][k] + g_vvx[i][j][k] + g_vvc[i][j][k];
				}
			}
		}
	}

	function poisson(iterMax) {
		const nnx=g_NNx,nny=g_NNy,nnz=g_NNz, h2=4.0*3.14159*g_dx*g_dx;
		const vvh=g_vvh,rho=g_rho;
		const w = (1.0/6.0)*1.8; // 1/4 * SOR omega(1.0<omega<2.0)

		for (let iter=1; iter<iterMax; iter++) {
			for (let i=1; i<nnx-1; i++) {
				for (let j=1; j<nny-1; j++) {
					for (let k=1; k<nnz-1; k++) {
						vvh[i][j][k] = vvh[i][j][k]+w*(vvh[i+1][j][k]+vvh[i-1][j][k]+vvh[i][j+1][k]+vvh[i][j-1][k]
										+vvh[i][j][k+1]+vvh[i][j][k-1]-6.0*vvh[i][j][k] +h2*rho[i][j][k]);
					}
				}
			}
		}
	}

	// LDA :  J. P. Perdew and A. Zunger; Phys. Rev., B23, 5048 (1981)
	function setVxc() {
		const nnx=g_NNx,nny=g_NNy,nnz=g_NNz;

		const c1 = -0.984745022;
		for (let i=1; i<nnx-1; i++) {
			for (let j=1; j<nny-1; j++) {
				for (let k=1; k<nnz-1; k++) {
					const rh = g_rho[i][j][k];
					const rh3 = Math.pow(rh,0.33333333);
					g_vvx[i][j][k] = c1*rh3;

					const rs = 0.6204/(rh3+1.0e-20);
					if (rs>=1.0) {
						const sqrtrs = Math.sqrt(rs);
						const ec = -0.1423/(1.0+1.0529*sqrtrs+0.3334*rs);
						g_vvc[i][j][k] = ec*(1.0+1.22838*sqrtrs+0.4445*rs)/(1.0+1.0529*sqrtrs+0.3334*rs);
					} else {
						g_vvc[i][j][k] = -0.05837-0.0084*rs +(0.0311+0.00133*rs)*Math.log(rs);
					}
				}
			}
		}
	}

	function eeCorrelation(rh) { // no use
		const r = 0.6204/(Math.pow(rh,0.33333333)+1.0e-20);
		let ec;
		if (r>=1.0) {
			ec = -0.1423/(1.0+1.0529*Math.sqrt(r)+0.3334*r);
		} else {
			ec = -0.0480-0.0116*r+(0.0311+0.0020*r)*Math.log(r);
		}
		return ec;
	}


	// --- sort srate

	function sortState(maxState){
		let iw, w;
		for (let iState=0; iState<maxState-1; iState++) {
			if (g_energy[iState]>g_energy[iState+1]+0.0001) {
				iw = g_psis[iState][0]; g_psis[iState][0]=g_psis[iState+1][0]; g_psis[iState+1][0]=iw;
				iw = g_psis[iState][1]; g_psis[iState][1]=g_psis[iState+1][1]; g_psis[iState+1][1]=iw;
				w = g_occ[iState]; g_occ[iState]=g_occ[iState+1]; g_occ[iState+1]=w;
				w = g_energy[iState]; g_energy[iState]=g_energy[iState+1]; g_energy[iState+1]=w;
			}
		}
	}


	// --- set loss

	function setLoss(damp) {
		const norb=g_numberOfOrbit;

		for (let iob=0; iob<norb; iob++) {
			steepestDescent(g_psi[iob], g_vv, damp);
		}
	}

	/*
	function setLoss(damp) {
		const nst=g_numberOfState;

		for (let ist=0; ist<nst; ist++) {
			const iob = g_psis[ist][0], spin = g_psis[ist][1];
			if (spin==1) {
				steepestDescent(g_psi[iOrbit], vvUp, damp);
			} else if (spin==-1) {
				steepestDescent(g_psi[iOrbit], vvDown, damp);
			}
		}
	}
	*/

	function steepestDescent(ph, v, damp) { // ph[][][][], v[][][]
		const nnx=g_NNx,nny=g_NNy,nnz=g_NNz, h2=2.0*g_dx*g_dx;
		const wrk=g_wrk; // wrk[][][][]

		const ee = kineticEnergy(ph) + potentialEnergy(ph,v);

		// wrk = (H - ee)ph
		for (let i=1; i<nnx-1; i++) {
			for (let j=1; j<nny-1; j++) {
				for (let k=1; k<nnz-1; k++) {
					wrk[i][j][k][0] = -(ph[i+1][j][k][0]+ph[i-1][j][k][0]+ph[i][j+1][k][0]+ph[i][j-1][k][0]
						+ph[i][j][k+1][0]+ph[i][j][k-1][0]-6.0*ph[i][j][k][0])/h2+(v[i][j][k]-ee)*ph[i][j][k][0];
					wrk[i][j][k][1] = -(ph[i+1][j][k][1]+ph[i-1][j][k][1]+ph[i][j+1][k][1]+ph[i][j-1][k][1]
						+ph[i][j][k+1][1]+ph[i][j][k-1][1]-6.0*ph[i][j][k][1])/h2+(v[i][j][k]-ee)*ph[i][j][k][0];
				}
			}
		}
		// ph = ph - damp*(H - ee)ph
		for (let i=1; i<nnx-1; i++) {
			for (let j=1; j<nny-1; j++) {
				for (let k=1; k<nnz-1; k++) {
					ph[i][j][k][0] -= damp*wrk[i][j][k][0];
					ph[i][j][k][1] -= damp*wrk[i][j][k][1];
				}
			}
		}

		normalize(ph);
	}


	// --- Gram-Schmidt

	function GramSchmidt(stateMax) {
		const nnx=g_NNx, nny=g_NNy, nnz=g_NNz, dv=g_dx*g_dy*g_dz;
		const psi=g_psi;

		normalize(psi[0]);

		for (let iState=1; iState<stateMax; iState++) {
			const iOrbit = g_psis[iState][0];
			for (let jState=0; jState<iState; jState++) {
				const jOrbit = g_psis[jState][0];
				if (g_psis[jState][1]==g_psis[iState][1]) {
					let pRe = 0.0, pIm = 0.0;
					for (let i=1; i<nnx-1; i++) {
						for (let j=1; j<nny-1; j++) {
							for (let k=1; k<nnz-1; k++) {
								pRe += (psi[jOrbit][i][j][k][0]*psi[iOrbit][i][j][k][0]
										+ psi[jOrbit][i][j][k][1]*psi[iOrbit][i][j][k][1])*dv;
								pIm += (psi[jOrbit][i][j][k][0]*psi[iOrbit][i][j][k][1]
										- psi[jOrbit][i][j][k][1]*psi[iOrbit][i][j][k][0])*dv;
							}
						}
					}
					for (let i=1; i<nnx-1; i++) {
						for (let j=1; j<nny-1; j++) {
							for (let k=1; k<nnz-1; k++) {
								psi[iOrbit][i][j][k][0] -= (pRe*psi[jOrbit][i][j][k][0]-pIm*psi[jOrbit][i][j][k][1]);
								psi[iOrbit][i][j][k][1] -= (pRe*psi[jOrbit][i][j][k][1]+pIm*psi[jOrbit][i][j][k][0]);
							}
						}
					}
				}
			}
			normalize(psi[iOrbit]);
		}
	}

	// --- set occupation

	function setOcc(maxState, nElectron) {

		let eUpper = g_energy[maxState-1];
		let eLower = g_energy[0];
		for (let i=0; i<maxState; i++) {
			if (g_energy[i]>eUpper) eUpper = g_energy[i];
			if (g_energy[i]<eLower) eLower = g_energy[i];
		}

		while (eUpper-eLower>1.0e-12) {
			const eFermi = (eUpper+eLower)/2.0;
			const ntrial = trialOcc(maxState, eFermi);
			if (ntrial<nElectron) {
				eLower = eFermi;
			} else {
				eUpper = eFermi;
			}
		}
		const eFermi = (eUpper+eLower)/2.0;

		for (let i=0; i<maxState; i++) {
			g_occ[i] = 1.0*FermiDirac(g_energy[i], eFermi);
			if (g_occ[i]<0.0001) g_occ[i] = 0.0;
			if (1.0-g_occ[i]<0.0001) g_occ[i] = 1.0;
		}
	}

	function trialOcc(maxState, eFermi) {
		let s = 0.0;
		for (let i=0; i<maxState; i++) {
			s += 1.0*FermiDirac(g_energy[i], eFermi);
		}
		return s;
	}

	function FermiDirac(ee, ef) {
		return ( 1.0/(Math.exp((ee-ef)/g_broadening)+1.0) );
	}

	function levelWidth() { // no use
		let kT = g_broadening;
		if (g_iterCount<30) {
			kT=0.1;
		} else {
			kT = 0.1-0.002*(g_iterCount-29);
			if (kT<g_broadening) kT = g_broadening;
		}
		return kT;
	}

	// --------------------  utility  --------------------

	function norm(ph) { // ph[][][][]
		const nnx=g_NNx, nny=g_NNy, nnz=g_NNz;

		let p=0.0;
		for (let i=1; i<nnx-1; i++) {
			for (let j=1; j<nny-1; j++) {
				for (let k=1; k<nnz-1; k++) {
					p += (ph[i][j][k][0]*ph[i][j][k][0]+ph[i][j][k][1]*ph[i][j][k][1]);
				}
			}
		}
		return p*g_dx*g_dy*g_dz;
	}

	function normalize(ph) { // ph[][][][]
		const nnx=g_NNx, nny=g_NNy, nnz=g_NNz;

		const a = Math.sqrt(norm(ph));
		for (let i=0; i<nnx; i++) {
			for (let j=0; j<nny; j++) {
				for (let k=0; k<nnz; k++) {
					ph[i][j][k][0] = ph[i][j][k][0]/a;
					ph[i][j][k][1] = ph[i][j][k][1]/a;
				}
			}
		}
	}

	function potentialEnergy(ph, v) { // ph[][][][], v[][][]
		const nnx=g_NNx, nny=g_NNy, nnz=g_NNz;

		let p=0.0;
		for (let i=1; i<nnx-1; i++) {
			for (let j=1; j<nny-1; j++) {
				for (let k=1; k<nnz-1; k++) {
					p += v[i][j][k]*(ph[i][j][k][0]*ph[i][j][k][0]+ph[i][j][k][1]*ph[i][j][k][1]);
				}
			}
		}
		return p*g_dx*g_dy*g_dz;
	}

	function kineticEnergy(ph) { // ph[][][][]
		const nnx=g_NNx, nny=g_NNy, nnz=g_NNz, h2=2*g_dx*g_dx;

		let p = 0.0;
		for (let i=1; i<nnx-1; i++) {
			for (let j=1; j<nny-1; j++) {
				for (let k=1; k<nnz-1; k++) {
					const d2phRe = -(ph[i+1][j][k][0]+ph[i-1][j][k][0]+ph[i][j+1][k][0]+ph[i][j-1][k][0]
								+ph[i][j][k+1][0]+ph[i][j][k-1][0]-6.0*ph[i][j][k][0])/h2;
					const d2phIm = -(ph[i+1][j][k][1]+ph[i-1][j][k][1]+ph[i][j+1][k][1]+ph[i][j-1][k][1]
								+ph[i][j][k+1][1]+ph[i][j][k-1][1]-6.0*ph[i][j][k][1])/h2;
					p += (ph[i][j][k][0]*d2phRe+ph[i][j][k][1]*d2phIm);
				}
			}
		}
		return p*g_dx*g_dy*g_dz;
	}

	function innerProduct(f, g) { // f[][][][], g[][][][]
		const nnx=g_NNx, nny=g_NNy, nnz=g_NNz, dv=g_dx*g_dy*g_dz;

		let pRe = 0.0, pIm = 0.0;
		for (let i=0; i<nnx; i++) {
			for (let j=0; j<nny; j++) {
				for (let k=0; k<nnz; k++) {
					pRe += (f[i][j][k][0]*g[i][j][k][0] + f[i][j][k][1]*g[i][j][k][1])*dv;
					pIm += (f[i][j][k][0]*g[i][j][k][1] - f[i][j][k][1]*g[i][j][k][0])*dv;
				}
			}
		}
		return Math.sqrt(pRe*pRe+pIm*pIm);
	}

	function meanPosX(ph) { // ph[][][][]
		const nnx=g_NNx, nny=g_NNy, nnz=g_NNz;

		let p=0.0;
		for (let i=1; i<nnx-1; i++) {
			for (let j=1; j<nny-1; j++) {
				for (let k=1; k<nnz-1; k++) {
					p += i*g_dx*(ph[i][j][k][0]*ph[i][j][k][0]+ph[i][j][k][1]*ph[i][j][k][1]);
				}
			}
		}
		return p*g_dx*g_dy*g_dz;
	}

	//

	function xPFlow(ph, i, j ,k) { // ph[][][][]
		const pxphRe = (ph[i+1][j][k][1]-ph[i-1][j][k][1])/(2*g_dx);
		const pxphIm = (-ph[i+1][j][k][0]+ph[i-1][j][k][0])/(2*g_dx);
		return (ph[i][j][k][0]*pxphRe + ph[i][j][k][1]*pxphIm);
	}

	function yPFlow(ph, i, j ,k) { // ph[][][][]
		const pyphRe = (ph[i][j+1][k][1]-ph[i][j-1][k][1])/(2*g_dy);
		const pyphIm = (-ph[i][j+1][k][0]+ph[i][j-1][k][0])/(2*g_dy);
		return (ph[i][j][k][0]*pyphRe + ph[i][j][k][1]*pyphIm);
	}

	function zPFlow(ph, i, j ,k) { // ph[][][][]
		const pzphRe = (ph[i][j][k+1][1]-ph[i][j][k-1][1])/(2*g_dz);
		const pzphIm = (-ph[i][j][k+1][0]+ph[i][j][k-1][0])/(2*g_dz);
		return (ph[i][j][k][0]*pzphRe + ph[i][j][k][1]*pzphIm);
	}

	function psiDensity(ph,i,j,k) {
		return (ph[i][j][k][0]*ph[i][j][k][0]+ph[i][j][k][1]*ph[i][j][k][1]);
	}

	function psiPhase(ph,i,j,k) { // 0...2PI
		return (Math.PI + Math.atan2(ph[i][j][k][1],ph[i][j][k][0])); // atan2(y,x)
	}

	function psiXCurrent(ph,i,j,k) {
		const pRe = (ph[i+1][j][k][1]-ph[i-1][j][k][1])/(2*g_dx);
		const pIm = (-ph[i+1][j][k][0]+ph[i-1][j][k][0])/(2*g_dx);
		return (ph[i][j][k][0]*pRe + ph[i][j][k][1]*pIm)*g_dx*g_dy*g_dz;
	}

	function psiYCurrent(ph,i,j,k) {
		const pRe = (ph[i][j+1][k][1]-ph[i][j-1][k][1])/(2*g_dy);
		const pIm = (-ph[i][j+1][k][0]+ph[i][j-1][k][0])/(2*g_dy);
		return (ph[i][j][k][0]*pRe + ph[i][j][k][1]*pIm)*g_dx*g_dy*g_dz;
	}

	function psiZCurrent(ph,i,j,k) {
		const pRe = (ph[i][j][k+1][1]-ph[i][j][k-1][1])/(2*g_dz);
		const pIm = (-ph[i][j][k+1][0]+ph[i][j][k-1][0])/(2*g_dz);
		return (ph[i][j][k][0]*pRe + ph[i][j][k][1]*pIm)*g_dx*g_dy*g_dz;
	}

	function psiOrbDensity(orb,i,j,k) {
		return psiDensity(g_psi[orb],i,j,k);
	}

	function psiOrbPhase(orb,i,j,k) {
		return psiPhase(g_psi[orb],i,j,k);
	}


	// --------------------  move nuclei
	//
	// g_auLength = 5.29177e-11; // (m) 1(au) = g_auLength (m)
	// g_auTime = 2.41888e-17;   // (s) 1(au) = g_auTime (s)
	// g_auEnergy = 4.38975e-18; // (J) 1(au) = g_auEnergy (J)
	// g_au2eV = 27.2114;        // (eV/au)  1(au) = 27.2114(eV)

	function setInitNuc(iNuc, ncMass, ncCharge, ncxx, ncyy, nczz, ncvx, ncvy, ncvz) {
		g_nucMass[iNuc] = ncMass*1.67e-27; // (kg)
		g_nucCharge[iNuc] = ncCharge;
		g_nucxx[iNuc] = ncxx; // i-th nuclear x-position in (au)
		g_nucyy[iNuc] = ncyy;
		g_nuczz[iNuc] = nczz;
		g_nucvx[iNuc] = ncvx; // i-th nuclear x-velocity in SI(m/s)
		g_nucvy[iNuc] = ncvy;
		g_nucvz[iNuc] = ncvz;
		g_nucfx[iNuc] = 0.0;  // i-th nuclear x-force in SI(N)
		g_nucfy[iNuc] = 0.0;
		g_nucfz[iNuc] = 0.0;
	}

	function moveNuc(deltat) {
		const nnuc=g_nucMax;

		const dtNuc = deltat*g_auTime; // au to SI unit

		for (let i=0; i<nnuc; i++) {
			const a = 0.5*dtNuc/g_nucMass[i];
			g_nucvx[i] += a*g_nucfx[i]; // in SI
			g_nucvy[i] += a*g_nucfy[i]; // in SI
			g_nucvz[i] += a*g_nucfz[i]; // in SI
			g_nucxx[i] += (g_nucvx[i]*dtNuc)/g_auLength; // in au
			g_nucyy[i] += (g_nucvy[i]*dtNuc)/g_auLength; // in au
			g_nuczz[i] += (g_nucvz[i]*dtNuc)/g_auLength; // in au
		}
		setNucForce();
		for (let i=0; i<nnuc; i++) {
			const a = 0.5*dtNuc/g_nucMass[i];
			g_nucvx[i] += a*g_nucfx[i]; // in SI
			g_nucvy[i] += a*g_nucfy[i]; // in SI
			g_nucvz[i] += a*g_nucfz[i]; // in SI
		}
		setNucBoundary();
		setNucleiPotential();
	}

	function setNucBoundary() {
		const nnuc=g_nucMax, xmax=g_NNx*g_dx, yMax=g_NNy*g_dy, zMax=g_NNz*g_dz;

		for (let i=0; i<nnuc; i++) {
			if (g_nucxx[i]<-1.0) { g_nucxx[i] = -1.0; g_nucvx[i] = -g_nucvx[i]; }
			if (g_nucxx[i]>xMax+1.0) {g_nucxx[i] = xMax+1.0; g_nucvx[i] = -g_nucvx[i]; }
			if (g_nucyy[i]<-1.0) { g_nucyy[i] = -1.0; g_nucvy[i] = -g_nucvy[i]; }
			if (g_nucyy[i]>yMax+1.0) { g_nucyy[i] = yMax+1.0; g_nucvy[i] = -g_nucvy[i]; }
			if (g_nuczz[i]<-1.0) { g_nuczz[i] = -1.0; g_nucvz[i] = -g_nucvz[i]; }
			if (g_nuczz[i]>zMax+1.0) { g_nuczz[i] = zMax+1.0; g_nucvz[i] = -g_nucvz[i]; }
		}
	}

	function setNucForce() {
		const nnuc=g_nucMax;

		for (let i=0; i<nnuc; i++) {
			setElectronForceAtNuc(i);
		}
		for (let i=0; i<nnuc; i++) {
			for (let j=i+1; j<nnuc; j++) {
				const xi = g_nucxx[i], xj = g_nucxx[j];
				const yi = g_nucyy[i], yj = g_nucyy[j];
				const zi = g_nuczz[i], zj = g_nuczz[j];
				const r2 = (xi-xj)*(xi-xj) + (yi-yj)*(yi-yj) + (zi-zj)*(zi-zj);
				let r = Math.sqrt(r2); if (r<0.5*g_dx) r = 0.5*g_dx;
				const f = (g_auEnergy/g_auLength)*g_nucCharge[i]*g_nucCharge[j]/(r*r);
				g_nucfx[i] += f*(xi-xj)/r;
				g_nucfy[i] += f*(yi-yj)/r;
				g_nucfz[i] += f*(zi-zj)/r;
				g_nucfx[j] -= f*(xi-xj)/r;
				g_nucfy[j] -= f*(yi-yj)/r;
				g_nucfz[j] -= f*(zi-zj)/r;
			}
		}
	}

	function setElectronForceAtNuc(iNuc) {
		const nnx=g_NNx, nny=g_NNy, nnz=g_NNz;

		let sx = 0.0, sy = 0.0, sz = 0.0;
		for (let i=0; i<nnx; i++) {
			for (let j=0; j<nny; j++) {
				for (let k=0; k<nnz; k++) {
					const x = i*g_dx, y = j*g_dy, z = k*g_dz;
					const r2 = (x-g_nucxx[iNuc])*(x-g_nucxx[iNuc]) + (y-g_nucyy[iNuc])*(y-g_nucyy[iNuc])
							+ (z-g_nuczz[iNuc])*(z-g_nuczz[iNuc]);
					let r = Math.sqrt(r2); if (r<g_dx) r = g_dx;
					const f = (g_auEnergy/g_auLength)*(-g_rho[i][j][k]*g_dx*g_dy*g_dz)*g_nucCharge[iNuc]/(r*r);
					sx += -f*(x-g_nucxx[iNuc])/r;
					sy += -f*(y-g_nucyy[iNuc])/r;
					sz += -f*(z-g_nuczz[iNuc])/r;
				}
			}
		}
		g_nucfx[iNuc] = sx;
		g_nucfy[iNuc] = sy;
		g_nucfz[iNuc] = sz;
	}

	//

	function setNucleiPotential() {
		const nnx=g_NNx, nny=g_NNy, nnz=g_NNz, nnuc=g_nucMax;

		for (let i=0; i<nnx; i++) {
			for (let j=0; j<nny; j++) {
				for (let k=0; k<nnz; k++) {
					g_vvext[i][j][k] = 0.0;
				}
			}
		}

		for (let iNuc=0; iNuc<nnuc; iNuc++) {
			addNucPotential(iNuc);
		}
	}

	function addNucPotential(iNuc) {
		const nnx=g_NNx, nny=g_NNy, nnz=g_NNz;

		let a = g_jelliumRadius;
		if (a<0.25) a = 0.25;
		const qq = g_nucCharge[iNuc]; // charge
		const xPos = g_nucxx[iNuc], yPos = g_nucyy[iNuc], zPos = g_nuczz[iNuc];

		for (let i=0; i<nnx; i++) {
			for (let j=0; j<nny; j++) {
				for (let k=0; k<nnz; k++) {
					const x = i*g_dx, y = j*g_dy, z = k*g_dz;
					const r = Math.sqrt((x-xPos)*(x-xPos)+(y-yPos)*(y-yPos)+(z-zPos)*(z-zPos));
					if (r>a) {
						g_vvext[i][j][k] += -qq/r;
					} else {
						g_vvext[i][j][k] += -(qq/a)*(1.5-0.5*(r*r/(a*a)));
					}
				}
			}
		}
	}


	// --------------------  public  --------------------

	return {
		init:			setInitialCondition,	// setInitialCondition( theme )
		evolve:			timeEvolution,			// timeEvolution( nCalc, lossSW, qmdSW )

		getAUinSI:		function() { return [ g_auLength, g_auTime, g_auEnergy, g_au2eV ]; },
		getSysParam:	function() { return [ g_NNx, g_NNy, g_NNz, g_dx, g_timeStep, g_numberOfState ]; },
		getNow:			function() { return [ g_iterCount, g_sysTime ]; },

		getOccupation:	function(ist) { return g_occ[ist]; },
		getStEnergy:	function(ist) { return g_energy[ist]; },
		getState:		function(ist) { return [ g_psis[ist][0], g_psis[ist][1] ]; }, // [ orbit, spin ]
		getPsi:			function(orbit,i,j,k) { return g_psi[orbit][i][j][k]; }, // [ RePsi, ImPsi ]
		getOrbPhase:	function(orbit,i,j,k) { return psiPhase(g_psi[orbit],i,j,k); },
		getOrbDensity:	function(orbit,i,j,k) { return psiDensity(g_psi[orbit],i,j,k); },
		getOrbCurrent:	function(orbit,i,j,k) { const ph = g_psi[orbit];
							return [ psiXCurrent(ph,i,j,k), psiYCurrent(ph,i,j,k), psiZCurrent(ph,i,j,k) ]; },
		getDensity:		function(i,j,k) { return g_rho[i][j][k]; },

		getVext:		function(i,j,k) { return g_vvext[i][j][k]; },
		getVeff:		function(i,j,k) { return g_vv[i][j][k]; },
		getVh:			function(i,j,k) { return g_vvh[i][j][k]; },
		getVx:			function(i,j,k) { return g_vvx[i][j][k]; },
		getVc:			function(i,j,k) { return g_vvc[i][j][k]; },
	};

})(); // ====================  electronsTDKS3D end  ====================


const js100 = (function(){ // ====================  js Module  ====================

	const theModule = electronsTDKS3D;
	const xCanvasSize = 480;	// in pixel
	const yCanvasSize = 480;	// in pixel
	let canvas;					// canvas2d
	let ctx;					// = canvas.getContext('2d');

	let v_theme = 0;	// 0:(up,up) 1:(up,down) 2:(up,up,down) 3:(up,up,up,up)
	let v_nCalc = 2;	// n-times evolve time step
	let v_lossSW = 0;	// 0-lossOFF 1-lossON 2-loss+trans 3-loss+trans,t-stop
	let v_qmdSW = 0;	// QMD(nuclear motion) 0:off 1:on

	let p_auLength, p_auTime, p_auEnergy, p_au2eV; // <-- theModule.getAUinSI();
	let p_NNx, p_NNy, p_NNz, p_dx, p_timeStep, p_numberOfState; // <-- theModule.getSysParam();

  let iterCount, sysTime;
  let occList = [];
  let stEnergyList = [];
	let spinList =[];

	let nowData = [];
  let densityArray = [];
	let vextArray = [];

	let dispMode = 1;
	let dispState = 0;
	let resetFlag = true;
	let pauseFlag = false;
	let stepFlag = false;
	let inStepFlag = false;

  let breakFlag = false;
  let getFieldFlag = true;
  let fieldKind = 1;


	function main() {
		resetFlag = true;
		setCanvas( 'canvas_box', xCanvasSize, yCanvasSize );
		initDom();
		viewHome();

		animate();

		function setCanvas( canvasID, width, height ) {
			canvas = document.getElementById( canvasID );
			canvas.width  = width;
			canvas.height = height;
			ctx = canvas.getContext('2d');
			ctx.font = "16px 'sans-serif'";
			ctx.textBaseline = "bottom";
			ctx.textAlign = "left";
			ctx.lineWidth = 1;
			g3d.setMouseOnCanvas( canvas ); // 3D graphics
		}
	}


	function animate() {
    if ( breakFlag ) return;

		if ( resetFlag ) {
			resetFlag = false;
			theModule.init( v_theme ); // ( nn, BoxSizeInNM, contTemp )
			[ p_NNx, p_NNy, p_NNz, p_dx, p_timeStep, p_numberOfState ] = theModule.getSysParam();
			v_lossSW = 0;
			document.getElementById("slct_loss").selectedIndex = 0;
			changeStateSelectorOptions( p_numberOfState );

			// g3d.init( NNx, NNy, NNz, dx, xCanvasSize, yCanvasSize, xBoxSize, yShift );
			g3d.init( p_NNx, p_NNy, p_NNz, p_dx, xCanvasSize, yCanvasSize, 300, 20 );
			g3d.drawGrid2D.threshold = 0.02;
      getFieldFlag = true;
      fieldKind = 1;
		}

		if ( !pauseFlag ) {
			theModule.evolve( v_nCalc, v_lossSW, v_qmdSW );
		} else if ( pauseFlag && stepFlag ) {
			stepFlag = false;
			theModule.evolve( v_nCalc, v_lossSW, v_qmdSW );
			inStepFlag = true;
		}

		draw( ctx, dispMode );

    if ( getFieldFlag ) setFieldData( fieldKind );

		requestAnimationFrame(animate);
	}

  function setFieldData( fieldKind ) {
    for (let ist=0; ist<p_numberOfState; ist++) {
      occList[ist] = theModule.getOccupation(ist);
      stEnergyList[ist] = theModule.getStEnergy(ist);
      spinList[ist] = theModule.getState(ist)[1];
    }

    if (fieldKind==1) {
			densityArray = [];
			vextArray = [];
			nowData = [ iterCount, sysTime, occList, stEnergyList, spinList ];
      for (let i=0; i<p_NNx; i++) {
        densityArray[i] = [];
        vextArray[i] = [];
        for (let j=0; j<p_NNy; j++) {
          densityArray[i][j] = [];
          vextArray[i][j] = [];
          for (let k=0; k<p_NNz; k++) {
            densityArray[i][j][k] = theModule.getDensity(i,j,k);
            vextArray[i][j][k] = theModule.getVext(i,j,k);
          }
        }
      }
    }
  }


	// --------------------  draw  --------------------

	const gColor = { orb:"#dddd00", dens:"#dd88dd",
					Vext:"#00dd00", Veff:"#0088ff", Vh:"#4444ff", Vxc:"#8800ff", Vx:"#aa00ff",Vc:"#8888ff",
					text:"#888888", box:"#999900" };

	function draw( ctx, dispMode ) {

		const kc = p_NNz/2;

		const dispOcc = theModule.getOccupation( dispState );
		let dispOrbit, dispSpin; [ dispOrbit, dispSpin ] = theModule.getState( dispState );
		const dispSpinStr = (dispSpin==1) ? "up" : "down";

		// clear
		ctx.clearRect( 0, 0, xCanvasSize, yCanvasSize );

		if ( dispMode==0 ) {
			dispText( "state table" );
			drawStateTable( ctx );

		} else if ( dispMode==1 ) {
			dispText( "3D density(x,y,z)" );
			const densFunc = function(i,j,k) { return 1000.0*theModule.getDensity(i,j,k); };
			// drawField3D( ctx, rotAngle, fieldFunc, colorMode )
			drawField3D( ctx, 0.0, densFunc, 1 );

		} else if ( dispMode==2 ) {
			dispText( "3D state density(x,y,z)" );
			const orbFunc = function(i,j,k) { return 1000.0*theModule.getOrbDensity(dispOrbit,i,j,k); };
			// drawField3D( ctx, rotAngle, fieldFunc, colorMode );
			const colorMode = ( dispOcc>0.05 ) ? 1 : 2;
			drawField3D( ctx, 0.0, orbFunc, colorMode );

		}   else if ( dispMode==3 ) {
			dispText( "3D phase(x,y,z)" );
			const densFunc = function(i,j,k) { return 1000.0*theModule.getOrbDensity(dispOrbit,i,j,k); };
			const colorFunc = function(i,j,k) {
				const hue = theModule.getOrbPhase(dispOrbit, i,j,k)*180.0/Math.PI;
				return `hsl(${hue},${Math.floor(dispOcc*80+20)}%,50%)`;
			}
			// drawField3D( ctx, rotAngle, fieldFunc, colorFactor, scale, xPos, yPos )
			drawField3D( ctx, 0.0, densFunc, colorFunc );

		} else if ( dispMode==4 ) {
			dispText( "grid2d: (charge density+Vext)(x,y)" );
			const zFunc = function(i,j) { return 100.0*theModule.getDensity(i,j,kc) + 0.5*theModule.getVext(i,j,kc) };
			const colorFunc = function(i,j) {
				const z = 100.0*theModule.getDensity(i,j,kc);
				const hue = (18120-z*20.0)%360;
				return ( Math.abs(z/g3d.cz0)>0.02 ) ? "hsl("+(hue)+",100%,50%)" : "#004400";
			}
			// g3d.drawGrid2D( ctx, rotAngle, zFunc, colorFactor, inc [, showBox] )
			g3d.drawGrid2D( ctx, 0.0, zFunc, colorFunc, 1 );

		} else if (dispMode==5 ) {
			dispText( `grid2d: (state density+Vext)(x,y), |${dispState}>, spin:${dispSpinStr}, occ:${dispOcc}` );
			const zFunc = function(i,j) {
				return 100.0*theModule.getOrbDensity(dispOrbit,i,j,kc) + 0.5*theModule.getVext(i,j,kc) };
			const colorFunc = function(i,j) {
				const z = 100.0*theModule.getOrbDensity(dispOrbit,i,j,kc);
				const hue = (18120-z*20.0)%360;
				return ( Math.abs(z/g3d.cz0)>0.02 ) ? `hsl(${hue},${Math.floor(dispOcc*100)}%,50%)` : "#004400";
			}
			// g3d.drawGrid2D( ctx, rotAngle, zFunc, colorFactor, inc [, showBox] )
			g3d.drawGrid2D( ctx, 0.0, zFunc, colorFunc, 1 );

		}else if (dispMode==6 ) {
			dispText( `grid2d: (Re[state]+Vext)(x,y), |${dispState}>, spin:${dispSpinStr}, occ:${dispOcc}` );
			const zFunc = function(i,j) {
				return 30.0*theModule.getPsi(dispOrbit,i,j,kc)[0] + 0.5*theModule.getVext(i,j,kc) };
			const colorFunc = function(i,j) {
				const z = 30.0*theModule.getPsi(dispOrbit,i,j,kc)[0];
				const hue = (18120-z*20.0)%360;
				return ( Math.abs(z/g3d.cz0)>0.02 ) ? `hsl(${hue},${Math.floor(dispOcc*100)}%,50%)` : "#004400";
			}
			// g3d.drawGrid2D( ctx, rotAngle, zFunc, colorFactor, inc [, showBox] )
			g3d.drawGrid2D( ctx, 0.0, zFunc, colorFunc, 1 );

		} else if (dispMode==7 ) {
			dispText( `grid2d: (phase + Vext)(x,y), |${dispState}>, spin:${dispSpinStr}, occ:${dispOcc}` );
			const zFunc = function(i,j) {
				return 100.0*theModule.getOrbDensity(dispOrbit,i,j,kc) + 0.5*theModule.getVext(i,j,kc) };
			const colorFunc = function(i,j) {
				const z = 100.0*theModule.getOrbDensity(dispOrbit,i,j,kc);
				const hue = theModule.getOrbPhase(dispOrbit, i,j,kc)*180.0/Math.PI;
				return ( Math.abs(z/g3d.cz0)>0.02 ) ? `hsl(${hue},${Math.floor(dispOcc*100)}%,50%)` : "#004400";
			}
			// g3d.drawGrid2D( ctx, rotAngle, zFunc, colorFactor, inc [, showBox] )
			g3d.drawGrid2D( ctx, 0.0, zFunc, colorFunc, 1 );

		} else if (dispMode==8 ) {
			dispText( "external potential Vext(x,y)" );
			const zFunc = function(i,j) { return 0.5*theModule.getVext(i,j,kc); };
			// g3d.drawGrid2D( ctx, rotAngle, zFunc, colorFactor, inc [, showBox] )
			g3d.drawGrid2D( ctx, 0.0, zFunc, gColor.Vext, 1 );

		} else if (dispMode==9 ) {
			dispText( "effective potential Veff(x,y)" );
			const zFunc = function(i,j) { return 0.5*theModule.getVeff(i,j,kc); };
			// g3d.drawGrid2D( ctx, rotAngle, zFunc, colorFactor, inc [, showBox] )
			g3d.drawGrid2D( ctx, 0.0, zFunc, gColor.Veff, 1);

		} else if (dispMode==10 ) {
			dispText( "Hartree potential VH(x,y) x 10" );
			const zFunc = function(i,j) { return 5.0*theModule.getVh(i,j,kc); };
			// g3d.drawGrid2D( ctx, rotAngle, zFunc, colorFactor, inc [, showBox] )
			g3d.drawGrid2D( ctx, 0.0, zFunc, gColor.Vh, 1 );

		} else if (dispMode==11 ) {
			dispText( "exchange and correlation potential Vxc(x,y) x 10" );
			const zFunc = function(i,j) { return 5.0*(theModule.getVx(i,j,kc)+theModule.getVc(i,j,kc)); };
			// g3d.drawGrid2D( ctx, rotAngle, zFunc, colorFactor, inc [, showBox] )
			g3d.drawGrid2D( ctx, 0.0, zFunc, gColor.Vxc, 1 );

		}

		if ( dispMode>=1 ) drawOccupiedState( ctx );

		let auLength, auTime, auEnergy, au2eV
		[ auLength, auTime, auEnergy, au2eV ] = theModule.getAUinSI();
		[ iterCount, sysTime ] = theModule.getNow();

		ctx.fillStyle = gColor.text;
		ctx.fillText(`box = ${p_NNx*p_dx}x${p_NNy*p_dx}x${p_NNz*p_dx} (au)`, 20, yCanvasSize-5);
		ctx.fillText(`time = ${sysTime.toFixed(2)} (au)`, 240, yCanvasSize-5);


		function dispText( str ) {
			ctx.fillStyle = "#888888";
			ctx.fillText( str, 20, yCanvasSize-25 );
		}
	}


	function drawStateTable( ctx ) {
		const nst = p_numberOfState;
		const px = 15, py = 20;

		// clear canvas
		ctx.fillStyle = "#888888";
		ctx.fillRect( 5, 5, xCanvasSize-10, yCanvasSize-80 );

		ctx.fillStyle = "rgb(0,0,0)";
		ctx.fillText("State Table", px+50, py+20);
		ctx.fillText("state", px+30, py+60);
		ctx.fillText("state E (au)", px+90, py+60);
		ctx.fillText("occupation", px+190, py+60);
		ctx.fillText("orbNo.", px+290, py+60);
		ctx.fillText("spin", px+360, py+60);

		for (let ist=0; ist<nst; ist++) {
			const energy = theModule.getStEnergy( ist );
			const occ = theModule.getOccupation( ist );
			let orb, spin;
			[ orb, spin ] = theModule.getState( ist );

			const ppy = py+85+(nst-ist-1)*20;
			ctx.fillStyle = `hsl(${240-occ*240},100%,50%)`;
			ctx.fillText(`| ${ist} >`, px+30, ppy);
			ctx.fillText(`${energy.toFixed(6)}`, px+90, ppy);
			ctx.fillText(`${occ.toFixed(6)}`, px+190, ppy);
			ctx.fillText(`${orb}`, px+290, ppy);
			ctx.fillText(((spin==1) ? "up" : "down"), px+360, ppy);
		}
	}

	function drawOccupiedState( ctx ) {
		let ypp = 15;
		ctx.font = "12px 'sans-serif'";
		for (let ist=0; ist<p_numberOfState; ist++) {
			const energy = theModule.getStEnergy( ist );
			const occ = theModule.getOccupation( ist );
			if ( occ>0.1 ) {
				let orb, spin;
				[ orb, spin ] = theModule.getState( ist );
				const spinStr = (spin==1) ? "up" : "down";
				ctx.fillStyle = `hsl(${(ist*30)},${Math.floor(occ*80.0)+20}%,50%)`;
				ctx.fillText(`|${ist}>`, 150, ypp);
				ctx.fillStyle = "#aaaaaa";
				ctx.fillText(
					`energy:${energy.toFixed(6)},   occ:${occ.toFixed(4)},   spin:${spinStr}`,
					180, ypp);
				ypp +=15;
			}
		}
		ctx.font = "16px 'sans-serif'";
	}


	function drawField3D(ctx, rotAngle, fieldFunc, colorMode ) {
		let sc, xp, yp; [ sc, xp, yp ] = g3d.scxpypFunc();
		const nnx=g_NNx, nny=g_NNy, nnz=g_NNz, dx=g_dx, hh=g_dx*sc;
		const threshold = drawField3D.threshold;

		g3d.set3DRotateXY(rotAngle);
		g3d.plotFarEdge(ctx, sc,xp,yp,"#444400"); // dark yellow
		for (let ii=0; ii<nnx; ii++) {
			let i=ii; if (g3d.pzApex[1]-g3d.pzApex[0]<0) i=nnx-ii-1;
			for (let jj=0; jj<nny; jj++) {
				let j=jj; if (g3d.pzApex[2]-g3d.pzApex[0]<0) j=nny-jj-1;
				for (let kk=0; kk<nnz; kk++) {
					let k=kk; if (g3d.pzApex[4]-g3d.pzApex[0]<0) k=nnz-kk-1;

					const f = fieldFunc(i,nny-j-1,k);
					const r = Math.min(Math.pow(Math.abs(f),0.333),0.6*hh);
					if ( r>threshold ) {
						let th, colr;
						if ( colorMode==0 ) { // 0:red(f>0) or blue(f<0)
							colr = (f>0) ? "#ff0000" : "#0000ff";
						} else if ( colorMode==1 ) { //
							th = 120.0-30.0*r*Math.sign(f);
							if (th>270.0) th = 270.0;
							if (th<-30.0) th = -30.0;
							colr = `hsl(${th},100%,50%)`;
						} else if ( colorMode==2 ) { // small hsl3-arg.
							th = 120.0-30.0*r*Math.sign(f);
							if (th>270.0) th = 270.0;
							if (th<-30.0) th = -30.0;
							colr = `hsl(${th},60%,30%)`;
						} else if ( typeof(colorMode)=='function' ) {
							colr = colorMode(i,j,k);
						}
						g3d.drawRotatedDisc( ctx, i*dx, j*dx, k*dx, r*g3d.zoom, colr, sc, xp, yp );
					}
				}
			}
		}
		g3d.plotNearEdge( ctx, sc, xp, yp, "#999900"); //yellow

		ctx.font = "12px 'sans-serif'";
		ctx.fillStyle = "#888888";
		ctx.fillText(`Ax=${(g3d.Ax*180/Math.PI).toFixed(1)}, Ay=${(g3d.Ay*180/Math.PI).toFixed(1)}`, 10, 15);
		ctx.font = "16px 'sans-serif'";
	}
	drawField3D.threshold = 0.7;


	// --------------------  graphics 3D (field) module  --------------------
	//
	// ver 0.0.1  2018.12.16  last updated on 2023.03.01
	// ver 0.0.2  2023.03.03  last updated on 2023.06.01

	let g_NNx, g_NNy, g_NNz, g_dx, g_dy, g_dz, g_xCanvasSize, g_yCanvasSize, g_xBoxSize, g_yShift;

	const g3d = {};				// namespace of graphic 3D module

	g3d.mouseDownFlag = 0;		// 1:on mouse down, 0:else
	g3d.x_mouse = 0;			// x-position of mouse
	g3d.y_mouse = 0;			// y-position of mouse
	g3d.x0_mouse = 0;			// drag-started x-position of mouse
	g3d.y0_mouse = 0;			// drag-started y-position of mouse
	g3d.zoom = 1.0;

	g3d.xMax = 0.0;				// x-length of box
	g3d.yMax = 0.0;				// y-length of box
	g3d.zMax = 0.0;				// z-length of box
	g3d.cx0 = 0.0;				// x-component of rotate center
	g3d.cy0 = 0.0;				// y-component of rotate center
	g3d.cz0 = 0.0;				// z-component of rotate center
	g3d.Ax = -Math.PI/15.0;		// rotate angle around x-axis
	g3d.Ay = -Math.PI/15.0;		// rotate angle around y-axis
	g3d.ddAy = 0.0;				// Ay change rate for auto-rotate: eg. dday=0.5*Math.PI/180
	g3d.cosAx = 0.0;			// cosAx=Math.cos(Ax)
	g3d.sinAx = 0.0;			// sinAx=Math.sin(Ax)
	g3d.cosAy = 0.0;			// cosAy=Math.cos(Ay)
	g3d.sinAy = 0.0;			// sinAy=Math.sin(Ay)

	g3d.xApex = [0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0];
	g3d.yApex = [0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0];
	g3d.zApex = [0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0];
	g3d.pxApex = [0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0];
	g3d.pyApex = [0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0];
	g3d.pzApex = [0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0];
	g3d.boxApex = [[0,0,0], [1,0,0], [0,1,0], [1,1,0], [0,0,1], [1,0,1], [0,1,1], [1,1,1] ];
	g3d.boxEdge = [[0,1,9], [0,2,9], [0,4,9], [1,3,9], [1,5,9], [2,3,9],
					[2,6,9], [3,7,9], [4,5,9], [4,6,9], [5,7,9], [6,7,9] ];

	//--- set mouse on canvas

	// g3d.setMouseOnCanvas( canvas );
	g3d.setMouseOnCanvas = function( canvas ) {
		canvas.addEventListener('mousemove', g3d.mouse_move);
		canvas.addEventListener('mousedown', g3d.mouse_down);
		canvas.addEventListener('mouseup', g3d.mouse_up);
		//canvas.addEventListener("mousewheel", g3d.mouseWheel);
	};

	g3d.mouse_move = function(e) {
		const pi = Math.PI;

		if (g3d.mouseDownFlag==1) {
			g3d.x_mouse = e.clientX;
			g3d.y_mouse = e.clientY;
			g3d.Ay = g3d.Ay + 0.5*(g3d.x_mouse-g3d.x0_mouse)*pi/180;
			if (g3d.Ay<-pi) g3d.Ay += 2*pi;
			if (g3d.Ay>pi) g3d.Ay += -2*pi;
			//g3d.Ax = g3d.Ax + 0.5*(g3d.y_mouse-g3d.y0_mouse)*pi/180;
			g3d.Ax = g3d.Ax - 0.5*(g3d.y_mouse-g3d.y0_mouse)*pi/180;
			if (g3d.Ax<-0.5*pi) g3d.Ax = -0.5*pi;
			if (g3d.Ax>0.5*pi) g3d.Ax = 0.5*pi;
			g3d.x0_mouse = g3d.x_mouse;
			g3d.y0_mouse = g3d.y_mouse;
		}
	};

	g3d.mouse_down = function(e) {
		if (g3d.mouseDownFlag==0) {
			g3d.x0_mouse = e.clientX;
			g3d.y0_mouse = e.clientY;
			g3d.x_mouse = g3d.x0_mouse;
			g3d.y_mouse = g3d.y0_mouse;
			g3d.mouseDownFlag = 1;
		}
	};

	g3d.mouse_up = function(e) {
		if (g3d.mouseDownFlag==1) {
			g3d.mouseDownFlag = 0;
		}
	};

	g3d.mouseWheel = function(e) {
		g3d.deltaY = e.deltaY;
		if ( g3d.deltaY > 0 ) g3d.zoom *= 0.95;
		else if ( g3d.deltaY < 0 ) g3d.zoom *= 1.05;
		if ( g3d.zoom<0.5 ) g3d.zoom = 0.5;
		if ( g3d.zoom>2.0 ) g3d.zoom = 2.0;
	};

	//--- 3D graphics aid

	// g3d.init( NNx, NNy, NNz, dx, xCanvasSize, yCanvasSize, xBoxSize, yShift );
	g3d.init = function( NNx, NNy, NNz, dx, xCanvasSize, yCanvasSize, xBoxSize, yShift ) {
		g_NNx = NNx; g_NNy = NNy; g_NNz = NNz;
		g_dx = dx; g_dy = dx; g_dz = dx;
		g_xCanvasSize = xCanvasSize; g_yCanvasSize = yCanvasSize;
		g_xBoxSize = (xBoxSize==undefined) ? 300 : xBoxSize;
		g_yShift = (yShift==undefined) ? 20 : yShift;
		g3d.setSize();
	}

	g3d.setSize = function() {
		g3d.xMax = g_NNx*g_dx;		// x-length of box
		g3d.yMax = g_NNy*g_dy;		// y-length of box
		g3d.zMax = g_NNz*g_dz;		// z-length of box
		g3d.cx0 = 0.5*g3d.xMax;		// x-component of rotate center
		g3d.cy0 = 0.5*g3d.yMax;		// y-component of rotate center
		g3d.cz0 = 0.5*g3d.zMax;		// z-component of rotate center
	};

	// g3d.setRotateAngle( AxInDegree, AyInDegree );
	g3d.setRotateAngle = function( AxInDegree, AyInDegree ) {
		g3d.Ax = AxInDegree*Math.PI/180.0;
		g3d.Ay = AyInDegree*Math.PI/180.0;
	};

	// g3d.scxpypFunc();
	g3d.scxpypFunc = function() {
		const xBoxSize = g_xBoxSize;
		const xCenter = g_xCanvasSize/2, yCenter = g_yCanvasSize/2-g_yShift, yBoxSize = (xBoxSize/g_NNx)*g_NNy;
		const xp = xCenter - (xBoxSize/2)*g3d.zoom, yp = yCenter - (yBoxSize/2)*g3d.zoom; // g3d param
		const sc = xBoxSize/(g_NNx*g_dx)*g3d.zoom;
		return [ sc, xp, yp ];
	}

	g3d.set3DRotateXY = function(rotateRateOfAyInDegree) {
		g3d.ddAy = rotateRateOfAyInDegree*Math.PI/180.0;
		g3d.Ay= g3d.Ay + g3d.ddAy; // auto-rotate : if (ddAy==0.0), stop
		if (g3d.Ay>Math.PI) g3d.Ay = g3d.Ay - 2.0*Math.PI;
		if (g3d.Ay<-Math.PI) g3d.Ay = g3d.Ay + 2.0*Math.PI;
		g3d.setBox();           // set box apex
		g3d.setRotateXY(g3d.Ax,g3d.Ay); // set rotate param
		g3d.rotateApexXY();     // box Apex rotate--> pxApex[i],pyApex[i],pzApex[i]
		g3d.markFarEdge();      // boxEdge[iEdge][2]=1:far side edge or 0:near side edge
	};

	g3d.setBox = function() {
		for (let i=0; i<8; i++) {
			g3d.xApex[i] = g3d.boxApex[i][0]*g3d.xMax;
			g3d.yApex[i] = g3d.boxApex[i][1]*g3d.yMax;
			g3d.zApex[i] = g3d.boxApex[i][2]*g3d.zMax;
		}
	};

	g3d.setRotateXY = function(angleX,angleY) {
		g3d.cosAx = Math.cos(angleX);
		g3d.sinAx = Math.sin(angleX);
		g3d.cosAy = Math.cos(angleY);
		g3d.sinAy = Math.sin(angleY);
		g3d.cx0 = 0.5*g3d.xMax;
		g3d.cy0 = 0.5*g3d.yMax;
		g3d.cz0 = 0.5*g3d.zMax;
	};

	g3d.rotateApexXY = function() { // rotate box apex
		const cosAx=g3d.cosAx,sinAx=g3d.sinAx,cosAy=g3d.cosAy,sinAy=g3d.sinAy,cx0=g3d.cx0,cy0=g3d.cy0,cz0=g3d.cz0;

		for (let i=0; i<8; i++) {
			g3d.pxApex[i] = cosAy*(g3d.xApex[i]-cx0)+sinAy*(sinAx*(g3d.yApex[i]-cy0)+cosAx*(g3d.zApex[i]-cz0))+cx0;
			g3d.pyApex[i] = cosAx*(g3d.yApex[i]-cy0)-sinAx*(g3d.zApex[i]-cz0) + cy0;
			g3d.pzApex[i] =-sinAy*(g3d.xApex[i]-cx0)+cosAy*(sinAx*(g3d.yApex[i]-cy0)+cosAx*(g3d.zApex[i]-cz0))+cz0;
		}
	};

	g3d.markFarEdge = function() {
		//seek far apex --> iMin
		let zMin = g3d.pzApex[0];
		let iMin = 0;
		for (let i=1; i<8; i++) {
			if (zMin>g3d.pzApex[i]) {
				zMin = g3d.pzApex[i];
				iMin = i;
			}
		}
		//mark far edge
		for (let iEdge=0; iEdge<12; iEdge++) {
			g3d.boxEdge[iEdge][2] = 0;
			if (g3d.boxEdge[iEdge][0]==iMin || g3d.boxEdge[iEdge][1]==iMin) g3d.boxEdge[iEdge][2] = 1;
		}
	};

	g3d.drawRotatedDisc = function(ctx, x,y,z,r,color,sc,xp,yp) {
		const cosAx=g3d.cosAx,sinAx=g3d.sinAx,cosAy=g3d.cosAy,sinAy=g3d.sinAy,cx0=g3d.cx0,cy0=g3d.cy0,cz0=g3d.cz0;

		const x1 = cosAy*(x-cx0)+sinAy*(sinAx*(y-cy0)+cosAx*(z-cz0)) + cx0
		const y1 = cosAx*(y-cy0)-sinAx*(z-cz0) + cy0
		//z1 =-sinAy*(x-cx0)+cosAy*(sinAx*(y-cy0)+cosAx*(z-cz0)) + cz0
		g3d.drawDisc(ctx, x1*sc+xp,y1*sc+yp,r,color);
	};

	g3d.drawRotatedLine = function(ctx, x1,y1,z1,x2,y2,z2,color,sc,xp,yp) {
		const cosAx=g3d.cosAx,sinAx=g3d.sinAx,cosAy=g3d.cosAy,sinAy=g3d.sinAy,cx0=g3d.cx0,cy0=g3d.cy0,cz0=g3d.cz0;

		const x1p = cosAy*(x1-cx0)+sinAy*(sinAx*(y1-cy0)+cosAx*(z1-cz0)) + cx0
		const y1p = cosAx*(y1-cy0)-sinAx*(z1-cz0) + cy0
		const x2p = cosAy*(x2-cx0)+sinAy*(sinAx*(y2-cy0)+cosAx*(z2-cz0)) + cx0
		const y2p = cosAx*(y2-cy0)-sinAx*(z2-cz0) + cy0
		g3d.drawLine(ctx, x1p*sc+xp,y1p*sc+yp,x2p*sc+xp,y2p*sc+yp,color);
	};

	g3d.plotNearEdge = function(ctx, sc,xp,yp,color) {
		for (let iEdge=0; iEdge<12; iEdge++) {
			if (g3d.boxEdge[iEdge][2]==0) { //far edge mark = 1
				g3d.plotEdge(ctx, iEdge,sc,xp,yp,color);
			}
		}
	};

	g3d.plotFarEdge = function(ctx, sc,xp,yp,color) {
		for (let iEdge=0; iEdge<12; iEdge++) {
			if (g3d.boxEdge[iEdge][2]==1) { //far edge mark = 1
				g3d.plotEdge(ctx, iEdge,sc,xp,yp,color);
			}
		}
	};

	g3d.plotEdge = function(ctx, iEdge,sc,xp,yp,color) {
		let iApex = g3d.boxEdge[iEdge][0];
		const x1=g3d.pxApex[iApex]*sc+xp, y1=g3d.pyApex[iApex]*sc+yp;
		iApex = g3d.boxEdge[iEdge][1];
		const x2=g3d.pxApex[iApex]*sc+xp, y2=g3d.pyApex[iApex]*sc+yp;
		g3d.drawLine(ctx, x1, y1, x2, y2, color);
	};

	g3d.drawLine = function(ctx, x1, y1, x2, y2, color) {
		ctx.strokeStyle = color;
		ctx.beginPath();
		ctx.moveTo(x1, y1);
		ctx.lineTo(x2, y2);
		ctx.stroke();
	};

	g3d.drawDisc = function(ctx, x, y, r, color) {
		ctx.fillStyle = color;
		ctx.beginPath();
		ctx.arc(x, y, r, 0, 2*Math.PI, false);
		ctx.fill();
	};

	// --------------------  end of graphics 3D (field) module  --------------------


	// g3d_extension grid2d  created 2023.06.01, last updated 2023.06.04
	// g3d.drawGrid2D( ctx, rotAngle, zFunc, colorFactor, inc [, showBox] )
	g3d.drawGrid2D = function ( ctx, rotAngle, zFunc, colorFactor, inc, showBox ) {
		let sc, xp, yp; [ sc, xp, yp ] = g3d.scxpypFunc();
		const nnx = g_NNx, nny = g_NNy, threshold = g3d.drawGrid2D.threshold;

		g3d.set3DRotateXY(rotAngle);
		if ( (showBox & 1)>0 || showBox==undefined ) g3d.plotFarEdge(ctx, sc,xp,yp,"#444400"); // dark yellow
		for (let jj=0; jj<nny; jj+=inc) {
			let j=jj;if (g3d.pzApex[2]-g3d.pzApex[0]<0) j=nny-jj-1;
			for (let ii=0; ii<nnx; ii++) {
				let i=ii;if (g3d.pzApex[1]-g3d.pzApex[0]<0) i=nnx-ii-1;
				if (i<0 || i+1>nnx-1) continue;
				const f = zFunc(i,nny-j-1);
				const x = i*g_dx, y = j*g_dy, z = f + g3d.cz0;
				const f1 = zFunc(i+1,nny-j-1);
				const x1 = (i+1)*g_dx, y1 = j*g_dy, z1 = f1 + g3d.cz0;
				let colr;
				if ( typeof(colorFactor)=='number' ) {
					const th = (18120 - Math.floor(colorFactor*180.0*(f+f1)/g3d.cz0))%360;
					const a = Math.abs((f+f1)/g3d.cz0);
					colr = (a>threshold) ? `hsl(${th},100%,50%)` : "#444444" ;
				} else if ( typeof(colorFactor)=='string' ) {
					colr = colorFactor;
				} else if ( typeof(colorFactor)=='function' ) {
					colr = colorFactor(i,nny-j-1);
				}
				g3d.drawRotatedLine(ctx, x,y,z,x1,y1,z1,colr,sc,xp,yp);
			}
		}
		for (let ii=0; ii<nnx; ii+=inc) {
			let i=ii;if (g3d.pzApex[1]-g3d.pzApex[0]<0) i=nnx-ii-1;
			for (let jj=0; jj<nny; jj++) {
				let j=jj;if (g3d.pzApex[2]-g3d.pzApex[0]<0) j=nny-jj-1;
				if (j<0 || j+1>nny-1) continue;
				const f = zFunc(i,nny-j-1);
				const x = i*g_dx, y = j*g_dy, z = f + g3d.cz0;
				const f1 = zFunc(i,nny-j-2);
				const x1 = i*g_dx, y1 = (j+1)*g_dy, z1 = f1 + g3d.cz0;
				let colr;
				if ( typeof(colorFactor)=='number' ) {
					const th = (18120 - Math.floor(colorFactor*180.0*(f+f1)/g3d.cz0))%360;
					const a = Math.abs((f+f1)/g3d.cz0);
					colr = (a>threshold) ? `hsl(${th},100%,50%)` : "#444444" ;
				} else if ( typeof(colorFactor)=='string' ) {
					colr = colorFactor;
				} else if ( typeof(colorFactor)=='function' ) {
					colr = colorFactor(i,nny-j-1);
				}
				g3d.drawRotatedLine(ctx, x,y,z,x1,y1,z1,colr,sc,xp,yp);
			}
		}
		if ( (showBox & 2)>0 || showBox==undefined ) g3d.plotNearEdge(ctx, sc,xp,yp,"#999900"); //yellow

		ctx.font = "12px 'sans-serif'";
		ctx.fillStyle = "#888888";
		ctx.fillText(`Ax=${(g3d.Ax*180/Math.PI).toFixed(1)}, Ay=${(g3d.Ay*180/Math.PI).toFixed(1)}`, 10, 15);
		ctx.font = "16px 'sans-serif'";
	};
	g3d.drawGrid2D.threshold = 0.005;


	// --------------------  control  --------------------

	function initDom() {
		document.getElementById("step_button").style.visibility = "hidden";
		document.getElementById("slct_orbit").disabled = true;
	}

	function changeStateSelectorOptions( nSelect ) {
		const element = document.getElementById( "slct_orbit" );
		if ( element.length==nSelect ) return;

		element.length = 0; // delete all options
		for (let i=0; i<nSelect; i++) {
			const option = document.createElement("option");
 			option.text = i;
 			element.appendChild(option);
		}
		element.options[0].selected = true;
		dispState = 0;
	}

	function reset() { resetFlag = true; }

	function pause() {
		let btn = document.getElementById("pause_button");

		pauseFlag = ( pauseFlag==false );
		if ( pauseFlag==false ) btn.innerHTML = "pause"; else btn.innerHTML = "go";

		if ( pauseFlag==true ) {
			document.getElementById("step_button").style.visibility = "visible";
		} else {
			document.getElementById("step_button").style.visibility = "hidden";
		}
	}

	function step() { stepFlag = true; }

	function setTheme() {  // select theme
		v_theme = 0 + document.getElementById("slct_theme").selectedIndex;
		resetFlag = true;
	}

	function setLossSW() {  // select lossSW
		v_lossSW = 0 + document.getElementById("slct_loss").selectedIndex;
	}

	function setDispMode() {  // select dispMode
		dispMode = 0 + document.getElementById("slct_dispMode").selectedIndex;
		const element = document.getElementById("slct_orbit")
		element.disabled = ( dispMode==2 || dispMode==3 || dispMode==5 || dispMode==6  || dispMode==7 ) ? false : true;
	}

	function setOrbit() {  // select dispOrbit
		dispState = 0 + document.getElementById("slct_orbit").selectedIndex;
	}

	function viewHome() {
		g3d.setRotateAngle( 10.5, -16.5 );
		g3d.zoom = 1.0;
	}

  // function controlled by python

  function breakLoop() {
    breakFlag = true;
  }

  function pysetTheme( theme ) {
    v_theme = theme
    document.getElementById("slct_theme").selectedIndex = theme;
    resetFlag = true;
  }

  function pysetDispMode( mode ) {
    dispMode = mode;
    document.getElementById("slct_dispMode").selectedIndex = mode;
  }

  function pygetData( pyMsg ) {
    document.getElementById("text_from_python").innerHTML = pyMsg;
    return [ iterCount, sysTime ];
  }

  function pygetOrbitData() {
    return [ occList, stEnergyList, spinList ];
  }

  function pygetFieldData() {
		fieldKind = 0;
    return [ nowData, vextArray, densityArray ];
  }


	// --------------------  public  --------------------

	return {
		main:			main,			// main()

		reset:			reset,			// reset()
		pause:			pause,			// pause()
		step:			step,			// step()

		setTheme:		setTheme,		// setTheme()
		setLossSW:		setLossSW,		// setLossSW()

		setDispMode:	setDispMode,	// setDispMode()
		setOrbit:		setOrbit,		// setOrbit()
		viewHome:		viewHome,		// viewHome()

    breakLoop: breakLoop, // breakLoop();
    pysetTheme: pysetTheme, // pysetTheme( theme )
    pysetDispMode: pysetDispMode, // pysetDispMode( mode )
    pygetData: pygetData, // pygetData( pyMsg ) : return [ iterCount, sysTime ]
    pygetOrbitData: pygetOrbitData, // pygetOrbitData() : return [ occList, stEnergyList, spinList ]
    pygetFieldData: pygetFieldData, // ygetFieldData() : return [ nowData, vextArray, densityArray ]
	};

})(); // ====================  js100 module end  ====================


const js = js100;
//window.addEventListener('load', js.main );
js.main();


// %%%%%%%%%%%%%%%%%%%%  end of javaScript  %%%%%%%%%%%%%%%%%%%%

</script>

<style type="text/css">
    body { text-align:left; color:#000000; background-color:#fff8dd; }
</style>

</head>

<body>
<p>[js100]  electrons - time dependent Kohn-Sham (TDKS3D)</p>
<canvas ID="canvas_box" style="background-color: #000000;" width="480" height="480"></canvas>
<br>

<label>theme:</label>
<select id="slct_theme" onChange="js.setTheme()">
<option selected>2-electrons spin-up,up</option><option>2-electrons spin-up,down</option>
<option>3-electrons spin-up,up,down</option><option>4-electrons spin-up,up,up,up</option>
</select>
    <span style="margin-right: 20px;"></span>
in parabolic Vext(x) = k x^2
<br>

<label>loss:</label>
<select id="slct_loss" onChange="js.setLossSW()">
<option>off</option><option>on</option><option>on+trans</option>
<option>on+trans,time stop</option>
</select>
    <span style="margin-right: 160px;"></span>
<button onClick="js.reset()">reset</button>
    <span style="margin-right: 20px;"></span>
<button id="pause_button" onClick="js.pause()">pause</button>
    <span style="margin-right: 10px;"></span>
<button id="step_button" onClick="js.step()">step</button>
<br>

<label>disp mode:</label>
<select id="slct_dispMode" onChange="js.setDispMode()">
<option>state table</option><option selected>3D: charge density(x,y,z)</option>
<option>3D: state density(x,y)</option><option>3D: state phase (angle:Im(psi)/Re(psi))</option>
<option>grid2D: (charge density + Vext)(x,y)</option><option>grid2D: (state density + Vext)(x,y)</option>
<option>grid2D: (Re[state] + Vext)(x,y)</option><option>grid2D: (state phase + Vext)(x,y)</option>
<option>grid2D: Vext(x,y)</option><option>grid2D: Veff(x,y)</option>
<option>grid2D: Vh(x,y) x 10</option><option>grid2D: Vxc(x,y) x 10</option>
</select>
    <span style="margin-right: 20px;"></span>
<label>state:</label>
<select id="slct_orbit" onChange="js.setOrbit()">
<option selected>0</option><option>1</option><option>2</option><option>3</option><option>4</option>
<option>5</option><option>6</option><option>7</option>
</select>
<br>

<button onClick="js.viewHome()">return to initial view</button>
<br>

<p id="text_caption" ></p>
<hr width="480" align="left" color="#a0a0a0">
<button onClick="js.breakLoop()">animation break to END</button>
    <span style="margin-right: 50px;"></span> python msg:
<span id="text_from_python" ></span>
<br>

</body>
</html>


  ''')
  display(htm)
# end def


In [ ]:
# exec html-js code
exec_html_js()
print("--- push [animation break to END] button to end ---")

In [ ]:
# get data and print

import time

# exec html-js code
exec_html_js()
print("-- start --")

# get data and print
for i in range(10):
  [ iterCount, sysTime ] = eval_js( 'js.pygetData({})'.format(i) )
  print(f'i = {i:>2},  count = {iterCount:>4},  system time = {sysTime:>6.2f} (au)')
  time.sleep(2)

# get orbit occupation and energy , and print
[ occList, energyList, spinList ] = eval_js( 'js.pygetOrbitData()' )
print("")
print( "orbit list :", [i for i in range(len(occList))] )
print( "orbit occupation list :", occList )
print( "orbit energy list :", energyList )
print( "orbit spin list :", ['up' if spn>0 else 'down' for spn in spinList] )

# animation break to END
eval_js( 'js.breakLoop()' )
print("-- end --")

In [ ]:
# change theme, dispMode

import time

themeList = [
  '0: 2-electrons spin-up,up', '1: 2-electrons spin-up,down',
  '2: 3-electrons spin-up,up,down', '3: 4-electrons spin-up,up,up,up' ]
dispModeList = [
  '0: state table', '1: 3D: charge density(x,y,z)', '2: 3D: state density(x,y)', '3: 3D: state phase (angle:Im(psi)/Re(psi))',
  '4: grid2D: (charge density + Vext)(x,y)', '5: grid2D: (state density + Vext)(x,y)', '6: grid2D: (Re[state] + Vext)(x,y)',
  '7: grid2D: (state phase + Vext)(x,y)', '8: grid2D: Vext(x,y)', '9: grid2D: Veff(x,y)',
  '10: grid2D: Vh(x,y) x 10', '11: grid2D: Vxc(x,y) x 10' ]

# exec html-js code
exec_html_js()
print("-- start --")

# set theme
theme = 2 # '2: 3-electrons spin-up,up,down'
eval_js( 'js.pysetTheme({})'.format(theme) )
print("-- set theme :", themeList[theme], " --")

# chabge disp mode and get data and print
for dispMode in range(len(dispModeList)):
  # change dispMode
  print(f"  -- dispMode: {dispModeList[dispMode]} --")
  eval_js( 'js.pysetDispMode({})'.format(dispMode) )

  # get data and print
  [ iterCount, totalEnergy ] = eval_js( 'js.pygetData({})'.format(dispMode) )
  print(f'\t  count = {iterCount:>4},  system time = {sysTime:>6.2f} (au)')
  time.sleep(3)

# animation break to END
eval_js( 'js.breakLoop()' )
print("-- end --")

In [ ]:
# get field data

import time
import numpy as np

# exec html-js code
exec_html_js()
print("-- start --")

# get data and print
for i in range(10):
  [ iterCount, sysTime ] = eval_js( 'js.pygetData({})'.format(i) )
  print(f'i = {i:>2},  count = {iterCount:>4},  system time = {sysTime:>8.2f} (au)')
  time.sleep(2)

# get field data
print("-- get field data  --")
[ nowData, vextArray, densityArray ] = eval_js( 'js.pygetFieldData()' )
[ iterCount, sysTime, occList, stEnergyList, spinList ] = nowData
print(f'got count = {iterCount:>4},  system time = {sysTime:>8.2f} (au)')
print( "orbit list :", [i for i in range(len(occList))] )
print( "orbit occupation list :", occList )
print( "orbit energy list :", energyList )
print( "orbit spin list :", ['up' if spn>0 else 'down' for spn in spinList] )

# animation break to END
eval_js( 'js.breakLoop()' )
print("-- simulator stop --")

# convert to nampy array
print("-- convert to numpy array Dens --")
Vext = np.array(vextArray)
Dens = np.array(densityArray)
print(f"got count = {iterCount},  Vext shape :{Vext.shape} , Dens shape :{Dens.shape} ")


In [ ]:
# save field data

import numpy as np

packed_data = np.array([ vextArray, densityArray ])

# print data shape
print(f"--  packed_data array shape :{packed_data.shape} --")

# save
print("-- save data as 'js100_data.npy' --")
np.save( 'js100_data.npy', packed_data )

In [ ]:
# load data and set numpy array for plot

import numpy as np

# load data = np.array([ vextArray, densityArray ])
print("-- load packed data --")
loaded_data = np.load('js100_data.npy')

# set numpy array
print("-- set numpy data --")
Vext = loaded_data[0]
Dens = loaded_data[1]
print(f"Vext shape :{Vext.shape} , Dens shape :{Dens.shape} ")

In [ ]:
# Vext(x,y) image

import numpy as np
import matplotlib.pyplot as plt

def image2d(fld, titleStr):
  nz = len(fld[0][0])
  Z = fld[:,:,int(nz/2)].T
  im = plt.imshow(Z, origin='lower', cmap='jet' )
  plt.colorbar(im)
  plt.title(titleStr)
  plt.show()

image2d(Vext,"Vext(x,y)")

In [ ]:
image2d(Dens,"electron density(x,y)")

In [ ]:
# electron density(x,y,z) - scatter plot 3D / plotly version

import numpy as np
import plotly.graph_objects as go

# electron density Dens(x,y,z) scatter3d plot
nx, ny, nz = Dens.shape
h = 0.5
x = np.linspace(-h*nx, h*nx, nx)
y = np.linspace(-h*ny, h*ny, ny)
z = np.linspace(-h*nz, h*nz, nz)
xx, yy, zz = np.meshgrid(x, y, z)

fig = go.Figure(data=go.Scatter3d(
    x=yy.flatten(), y=xx.flatten(), z=zz.flatten(),
    mode='markers',
    marker=dict(
        size=np.sqrt(abs(Dens.flatten()))*30,
        color=Dens.flatten(),
        colorscale='Jet',
        opacity=0.3, ),
    ))

fig.update_layout(
    title="electron density(x,y,z)",
    width=800, height=800,
    scene = {
        'camera_eye': {"x": 0.5, "y": -2.0, "z": 0.5},
        "aspectratio": {"x": 1, "y": ny/nx, "z": nz/nx},
        },
    )


fig.show()

In [ ]:
# plot external potential Vext(x,y,0) - surface plot

import numpy as np
import matplotlib.pyplot as plt

loaded_data = np.load('js100_data.npy')
# loaded_data = np.array([ vextArray, densityArray ])
V = loaded_data[0]

nx, ny, nz = V.shape
Vext2d = V[ : , : , int(nz/2) ]

# density field / surface plot with adjusted aspect ratio
fig = plt.figure(figsize=(8,8))
ax = fig.add_subplot(projection='3d')

# Plot the surface
x = np.arange(-nx/2,nx/2)
y = np.arange(-ny/2,ny/2)
X, Y = np.meshgrid(x, y)
Z = Vext2d.T
ax.plot_surface(X,Y,Z, cmap='jet')

# Adjust aspect ratio
ax.set_box_aspect([1, 1, 1])  # [x, y, z] aspect ratios

plt.xlabel("x")
plt.ylabel("y")
plt.title("Vext(x, y, 0)")
plt.show()